<h1> Elo Merchant Category Recommendation :</h1>

Reference : Make Sense Out of Nonsense : ELO EDA

<h2>Business Problem/Problem Statement :</h2>

> Elo Merchant Category Recommendation is a Kaggle competition which is provided by Elo. As a payment Brand, providing offer promotions and discounts with merchants is a good marketing strategy . Elo needs to keep their customers so loyalty of the customers towards the brand is crucial. For Example, a customer using the Elo card with diverse merchants for a long time, this signifies the user's loyalty is high. To keep the customer as a subscriber, Elo can run different promotional campaign’s targets towards customers with the customer’s favorite or frequently used merchants. These personalized reward programs are planned by the owners of the company to retain existing customers and attract new customers. So, the frequency of using their payment brand should increase. Basically, These programs make the customer’s choice more strongly towards the usage of Elo. The Problem is to find a metric which has to reflect the cardholder’s loyalty with Elo payment brand. Here we have the loyalty score which is a numerical score calculated 2 months after the historical and evaluation period. Elo uses it for their business decision about their promotional campaign.


<h2>Dataset Overview :</h2>

The datasets are largely anonymized, and the meaning of the features are not elaborated. External data is allowed.

The problem has 5 datasets.

> **train.csv (201917, 6) :** It has 6 features, first_active_month, card-id, feature1, feature2, feature3 and target

> **test.csv (123623, 5) :** The test set has the same features as the train set without targets

> **historical_transactions.csv (29112361, 14) :** Contains up to 3 months worth of historical transactions for each card_id

> **merchants.csv (334696, 22):** Contains the transactions at new merchants(merchant_ids that this particular card_id has not yet visited) over a period of two months.

> **new_merchant_transactions.csv (1963031, 14) :** Two months’ worth of data for each card_id containing ALL purchases that card_id made at merchant_ids that were not visited in the historical data

In all these datasets, no text data/feature is present. We only have categorical and numerical features. Additionally, by looking at historical_transactions.csv and new_merchant_transactions.csv, we can find that the historical transactions are the transactions occurred before the "reference date" and new merchant transactions - the ones that occurred after the reference date (according to the 'month_lag' field, which is generously described as "month lag to reference date").

<h2>Mapping the real-world problem to Machine Learning problem :</h2>

> In terms of Machine Learning, we need a metric to measure up the customer's loyalty.A certain loyalty score is assigned for each of the card_id present in train data.

>**Input Features —** Cardholder’s Purchase history, usage time etc.

>**Target Variable —** Loyalty Score

>The Loyalty Score is the target variable for which the Machine Learning Model should be built to predict. **What is loyalty?** According to the Data_Dictionary.xlsx, **loyalty is a numerical score calculated 2 months after historical and evaluation period.** The Loyalty score depends on many aspects of the customers. The purchase history, usage time, merchant’s diversity, etc.  Loyalty scores are real-numbers, It directly gives us the intuition that we have to go for a supervised machine learning regression model to solve this problem where features are as our input in train data and output is real number value which is our predicted loyalty score.

<h2>Real-world constraints :</h2>

> The constraint is that the data which has been provided is not real-customer data. The Provided data is Anonymous and simulated, I think this is due to privacy and legal constraints. Simulated data sometimes has an artificially induced bias which will affect the prediction model performance. We have to deal with this specifically.

<h2>Performance Metric :</h2>

> Root mean square error(RMSE) is used to evaluate our predictions with actual loyalty score. We want our predicted loyalty score close to the actual score. So we need to have a lower RMSE score. This gives us the knowledge that on the basis of input features how close our model makes the predictions as compared to actual predictions.


**My understanding of the problem :**

* Based on the data in historical_transactions.csv, Elo picked new mechants to recommend for each card holder.
* The date when Elo began providing recommentations is called the 'reference date'.
* The recommended mechant data is not provided (so we don't figure out the recommendation algorithm Elo uses).
* After the reference date, for each card Elo gathered transaction history for all new merchants that appeared on the card.
* By comparing each card's new merchant activity and the secret list of the merchants recommended by Elo, the loyalty score was calculated.
* **The goal is to evaluate Elo's recommendation algorithm by trying to predict in which cases it's going to work well (yielding a high loyalty score) and in which cases - not (yielding a low loyalty score).**

In [ ]:
! nvidia-smi -L

## **Import all the libraries :**

In [1]:
import keras
from keras import callbacks
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import models, layers
# from keras.optimizers import RMSprop, SGD
from keras.optimizers import Adam, Adadelta
from keras.optimizers.schedules import ExponentialDecay
from keras.metrics import RootMeanSquaredError

#As shown: 
#1) No convolution layer.
#2) Make the input shape equals to your regression features numbers.
#3) The network ends with output layer with only 1 output and no activation (it will be a linear layer, applying an activation function would constrain the range the output can take)
#4) Compile the network with the mse loss function—mean squared error, the square of the difference between the predictions and the targets. This is a widely used loss function for regression problems.
#5) Make the metric to monitor during training as mean absolute error (MAE). It’s the absolute value of the difference between the predictions and the targets. 

In [2]:
import numpy as np
import pandas as pd
import warnings
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
# plt.style.use('ggplot')

import datetime
import seaborn as sns
sns.set_style("whitegrid")
import warnings 
warnings.simplefilter("ignore")

# For computing Variable Inflation Factor (VIF)
from statsmodels.stats.outliers_influence import variance_inflation_factor

import gc
import pickle

# Display columns
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_info_columns', 150)
print(pd.get_option("display.max_rows"), pd.get_option("display.max_columns"))
print(pd.get_option("display.max_info_rows"), pd.get_option("display.max_info_columns"))

## **Functions :**

In [3]:
## Reference: https://www.kaggle.com/rinnqd/reduce-memory-usage

def reduce_memory_usage(df, verbose=True):
  '''
  This function reduces the memory sizes of dataframe by changing the datatypes of the columns.
  Parameters
  df - DataFrame whose size to be reduced
  verbose - Boolean, to mention the verbose required or not.
  '''
  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_mem = df.memory_usage().sum() / 1024**2
  for col in df.columns:
      col_type = df[col].dtypes
      if col_type in numerics:
          c_min = df[col].min()
          c_max = df[col].max()
          if str(col_type)[:3] == 'int':
              if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                  df[col] = df[col].astype(np.int8)
              elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                  df[col] = df[col].astype(np.int16)
              elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                  df[col] = df[col].astype(np.int32)
              elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                  df[col] = df[col].astype(np.int64)
          else:
              c_prec = df[col].apply(lambda x: np.finfo(x).precision).max()
              if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max and c_prec == np.finfo(np.float16).precision:
                  df[col] = df[col].astype(np.float16)
              elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max and c_prec == np.finfo(np.float32).precision:
                  df[col] = df[col].astype(np.float32)
              else:
                  df[col] = df[col].astype(np.float64)
  end_mem = df.memory_usage().sum() / 1024**2
  if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
  return df

# Check for missing values : train_data['new_hist_first_buy'].isna().any()
def check_missing_values(df):
    cols_missing_values = []
    for col in df.columns:
        if df[col].isna().any():
            cols_missing_values.append(col)
            print(col)
    return cols_missing_values
            
def create_new_columns(name, aggs):
    # get the individual key from dictionary and the corresponding list of functions for this key
    # For example : 'purchase_amount' key for functions ['sum','max','min','mean','var']
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

# Intersection function
def intersection(list1, list2): 
    return list(set(list1) & set(list2))

# Load saved model
def load_model_from_picklefile(filename):
    infile = open(filename,'rb')
    loaded_model = pickle.load(infile)
    infile.close()
    return loaded_model

# Save trained model
def save_model_to_picklefile(filename, save_model):
    model_file = open(filename,'wb')
    pickle.dump(save_model, model_file)
    model_file.close()

# Create file for submission to Kaggle
def create_file_for_submission(filename, card_ids, final_predictions):    
    kaggle = pd.DataFrame({'card_id': card_ids, 'target': final_predictions})
    kaggle.to_csv(filename, index=False)

**Loading Data :**


In [4]:
train_data = pd.read_csv('../input/elo-merchant-category-recommendation/train.csv')
test_data = pd.read_csv('../input/elo-merchant-category-recommendation/test.csv')
historical_data = pd.read_csv('../input/elo-merchant-category-recommendation/historical_transactions.csv')
newmerchant_data = pd.read_csv('../input/elo-merchant-category-recommendation/new_merchant_transactions.csv')
merchants_data = pd.read_csv('../input/elo-merchant-category-recommendation/merchants.csv')

In [5]:
train_data.shape, test_data.shape, historical_data.shape, newmerchant_data.shape, merchants_data.shape

**Reduce memory usage of data :**

In [6]:
train_data = reduce_memory_usage(train_data)
test_data = reduce_memory_usage(test_data)
historical_data = reduce_memory_usage(historical_data)
newmerchant_data = reduce_memory_usage(newmerchant_data)
merchants_data = reduce_memory_usage(merchants_data)

## (A) Data Preprocessing and Feature Engineering (Exploratory Data Analysis)

### Examining the feature statistics :

In [ ]:
plt.bar([0,1],[train_data.shape[0],test_data.shape[0]])
plt.xticks([0,1],['train_rows','test_rows'])

print('The number of rows in train_data is:',train_data.shape[0])
print('The number of rows in test_data is:',test_data.shape[0])

In [ ]:
train_data.info()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (15, 5));
train_data['feature_1'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='feature_1', rot=0);
train_data['feature_2'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='feature_2', rot=0);
train_data['feature_3'].value_counts().sort_index().plot(kind='bar', ax=ax[2], color='gold', title='feature_3', rot=0);
plt.suptitle('Counts of categories for train features');

fig, ax = plt.subplots(1, 3, figsize = (15, 5));
test_data['feature_1'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='feature_1', rot=0);
test_data['feature_2'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='feature_2', rot=0);
test_data['feature_3'].value_counts().sort_index().plot(kind='bar', ax=ax[2], color='gold', title='feature_3', rot=0);
plt.suptitle('Counts of categories for test features');

In [ ]:
# One-hot encode nominal feature_1, feature_2, feature_3
ohe_train_df_1 = pd.get_dummies(train_data['feature_1'], prefix='f1')
ohe_train_df_2 = pd.get_dummies(train_data['feature_2'], prefix='f2')
ohe_train_df_3 = pd.get_dummies(train_data['feature_3'], prefix='f3')

ohe_test_df_1 = pd.get_dummies(test_data['feature_1'], prefix='f1')
ohe_test_df_2 = pd.get_dummies(test_data['feature_2'], prefix='f2')
ohe_test_df_3 = pd.get_dummies(test_data['feature_3'], prefix='f3')

ohe_test_df_3

### Check for missing values in the loaded data frames

In [ ]:
check_missing_values(train_data), check_missing_values(test_data)

In [ ]:
check_missing_values(historical_data), check_missing_values(newmerchant_data)

In [ ]:
# for card_id[0], 'C_ID_0ab67a22ab' : there are 1304310 - 1304243 = 67 historical transactions
# for card_id[10], 'C_ID_4859ac9ed5' : there are 23622204 - 23622180 = 24 historical transactions
# historical_data.loc[23622180:23622204]

In [ ]:
numeric_feature_count = 0
non_numeric_feature_count = 0
numeric_features = []
non_numeric_features = []

for col in historical_data.columns:
  if (historical_data[col].dtypes == 'object'):
    non_numeric_feature_count += 1
    non_numeric_features.append(col)
    print("Non-numeric feature No. {} and name : {}".format(non_numeric_feature_count, col))
    print("No. of Missing values : {}, Zero values : {} with Mode value : {}".\
          format(historical_data[col].isnull().sum(), (historical_data[col] == 0).sum(),
                 historical_data[col].mode().values[0]))
    print("*" * 82)
    print()
  else:
    numeric_feature_count += 1
    numeric_features.append(col)
    print("Numeric feature No. {} and name : {}".format(numeric_feature_count, col))
    # (historical_data[col] == historical_data[col].median()).sum()
    print("No. of Missing values : {}, Zero values : {} with Mode value : {}".\
          format(historical_data[col].isnull().sum(), (historical_data[col] == 0).sum(),
                 historical_data[col].mode().values[0])) # Get only the value without the index
    print("-" * 82)
    print()
    

print("Total No. of Numeric features {} and Non-numeric features : {}".format(numeric_feature_count, non_numeric_feature_count))
print("\nNumeric features : {} and \nNon-numeric features : {}".format(numeric_features, non_numeric_features))

In [ ]:
historical_data['category_2'].value_counts()

## Impute missing values (with mean, median, mode) :
* For historical and new merchant data with mode values

In [7]:
# historical_data['category_2'].value_counts()
# historical_data['category_2'].unique() = [1., nan,  3.,  5.,  2.,  4.]
historical_data_cat2_mode = historical_data['category_2'].mode()
historical_data_merchant_id_mode = historical_data['merchant_id'].mode()
# historical_data['category_3'].value_counts()
historical_data_cat3_mode = historical_data['category_3'].mode()

print("historical_data mode for category_2 : {}".format(historical_data_cat2_mode[0]))
print("historical_data mode for merchant_id : {}".format(historical_data_merchant_id_mode[0]))
print("historical_data mode for category_3 : {}".format(historical_data_cat3_mode[0]))

# Replace missing values with mode values for category_3', 'merchant_id' and 'category_2'
# When inplace = True, the data is modified in place,
# which means it will return nothing and the dataframe is now updated.
for df in [historical_data, newmerchant_data]:
    df['category_2'].fillna(historical_data_cat2_mode[0], inplace = True)
    df['merchant_id'].fillna(historical_data_merchant_id_mode[0], inplace = True)
    df['category_3'].fillna(historical_data_cat3_mode[0], inplace = True)

### Generalization for 'purchase date' in 'historical data' and 'new merchant data'

In [ ]:
historical_data['purchase_date'].describe()

In [ ]:
historical_data['purchase_date'].min(), historical_data['purchase_date'].max(),\
historical_data['purchase_date'].mode()[0]

In [ ]:
newmerchant_data['purchase_date'].min(), newmerchant_data['purchase_date'].max(),\
newmerchant_data['purchase_date'].mode()[0]

In [ ]:
type(historical_data['purchase_date'].min()), type(historical_data['category_2'].mode()[0]) # type(newmerchant_data_purchase_date_min),

In [ ]:
# Check for missing values
check_missing_values(historical_data), check_missing_values(newmerchant_data)

In [ ]:
## Date operations
# datetime.datetime.today()
# (datetime.datetime.today() - newmerchant_data['purchase_date'][0]).days
# newmerchant_data['purchase_date'][0]
# type((datetime.datetime.today() - newmerchant_data['purchase_date']).dt.days)

# ((datetime.datetime.today() - df['purchase_date'][0]).dt.days)
# ((datetime.datetime.today() - newmerchant_data['purchase_date']).dt.days) // 30

# historical_data['purchase_amount'].describe()

# Range of purchase_date : Timestamp('2017-01-01 00:00:08'), Timestamp('2018-02-28 23:59:51')
# min_date = historical_data['purchase_date'].min() # historical_data['purchase_date'].max()
# pd.DatetimeIndex(historical_data['purchase_date']).astype(np.int64) * 1e-9
# historical_data['purchase_date'][0]

## Observe outliers and replaced with median values :
* For historical and new merchant data with mode values

In [ ]:
print('Quantile values for purchase amount in Historical Transaction :')
print('25th Percentile :',historical_data['purchase_amount'].quantile(0.25))
print('50th Percentile :',historical_data['purchase_amount'].quantile(0.50))
print('75th Percentile :',historical_data['purchase_amount'].quantile(0.75))
print('100th Percentile :',historical_data['purchase_amount'].quantile(1))
print('\n******************************************************************\n')
print('Quantile values for purchase amount in New Merchant Transaction :')
print('25th Percentile :',newmerchant_data['purchase_amount'].quantile(0.25))
print('50th Percentile :',newmerchant_data['purchase_amount'].quantile(0.50))
print('75th Percentile :',newmerchant_data['purchase_amount'].quantile(0.75))
print('100th Percentile :',newmerchant_data['purchase_amount'].quantile(1))

In [8]:
historical_data[historical_data['purchase_amount']  == 6010603.9717525]

In [9]:
historical_data_outlier_index = historical_data.loc[(historical_data['purchase_amount']  == 6010603.9717525)].index.values
print("The index of purchase amount in historical data :", historical_data_outlier_index)

In [10]:
newmerchant_data[newmerchant_data['purchase_amount']  == 263.15749789]

In [11]:
newmerchant_data_outlier_index = newmerchant_data.loc[(newmerchant_data['purchase_amount']  == 263.15749789), 'purchase_amount'].index.values
print("The index of purchase amount in newmerchant data :", newmerchant_data_outlier_index)

In [12]:
print("Original 'Purchase amount' in historical data :", historical_data.loc[historical_data_outlier_index[0], 'purchase_amount'])
print("Original 'Purchase amount' in newmerchant data :", newmerchant_data.loc[newmerchant_data_outlier_index[0], 'purchase_amount'])

# Replace outlier 'purchase amount' with median in historical data
historical_data.loc[historical_data_outlier_index[0], 'purchase_amount'] = historical_data['purchase_amount'].median()
# Replace outlier 'purchase amount' with median in newmerchant data
newmerchant_data.loc[newmerchant_data_outlier_index[0], 'purchase_amount'] = newmerchant_data['purchase_amount'].median()

print("New median 'Purchase amount' in historical data :", historical_data.loc[historical_data_outlier_index[0], 'purchase_amount'])
print("New median 'Purchase amount' in newmerchant data :", newmerchant_data.loc[newmerchant_data_outlier_index[0], 'purchase_amount'])

## Impute missing value in test data :
* Replace with the earliest purchase date

In [ ]:
# df_test['first_active_month'].isna().value_counts()
# df_test.loc[df_test['first_active_month'].isna()]
# idx_nan = df_test.loc[df_test['first_active_month'].isna()].index
# idx_nan.values

In [13]:
# test_data['first_active_month'].isna().value_counts()
missing_card_index = test_data.loc[test_data['first_active_month'].isna(), 'card_id'].index.values
print("The index of missing card in test data :", missing_card_index)

test_data.loc[test_data['first_active_month'].isna()]

In [14]:
missing_card_id = test_data.loc[test_data['first_active_month'].isna(), 'card_id'].reset_index(drop=True)[0]
# get the historical data for the missing card_id
card_missing_first_active_month = historical_data.loc[historical_data['card_id'] == missing_card_id]

print("Card_id : {} with {} transactions.".format(missing_card_id,
                                                  card_missing_first_active_month.shape[0]))

In [ ]:
# card_missing_first_active_month
# card_missing_first_active_month.columns
# Earliest 'purchase_date' for card_id 'C_ID_c27b4f80f7' is '2017-03-09'
# card_missing_first_active_month.sort_values(by=['purchase_date']).iloc[0]
# card_missing_first_active_month.sort_values(by=['purchase_date']).iloc[0:3, 10] # first three rows in position 10
# card_missing_first_active_month['purchase_date'].sort_values()
# card_missing_first_active_month['purchase_date'].value_counts()

# 55 transactions with card_id = 'C_ID_c27b4f80f7' that is without first_active_month
# historical_data.loc[historical_data['card_id'] == 'C_ID_c27b4f80f7', 'purchase_date']

In [15]:
card_missing_first_active_month['purchase_date'].min(), card_missing_first_active_month['purchase_date'].max()

In [16]:
# Fill the missing 'first_active_month' in test_data with the earliest value from 'purchase_date' with the card_id
# Fill the missing value with the mode value '2017-09' in 'first_active_month' of test_data
# df_test.loc[missing_card_index[0], 'first_active_month'] = '2017-03'
# df_test.loc[idx_nan.values]['first_active_month'] = '2017-09'
# df_test.loc[idx_nan.values]['first_active_month']

# format : YYYY-MM
test_data['first_active_month'].fillna('2017-03',inplace=True)
test_data.loc[missing_card_index[0]]['first_active_month']

In [17]:
test_data.loc[test_data['card_id'] == 'C_ID_c27b4f80f7'] # 2017-09

In [18]:
# Make sure no missing values in df_test
# df_test['first_active_month'].isna().value_counts()
check_missing_values(test_data)

In [ ]:
train_data['first_active_month'].min(), train_data['first_active_month'].max()

In [ ]:
train_data['first_active_month'].describe()

In [ ]:
train_data['first_active_month'] = pd.to_datetime(train_data['first_active_month'],
                                                  format='%Y-%m')

In [ ]:
plt.figure(figsize = (13,5))

sns.lineplot(x = train_data['first_active_month'], y= train_data['target'])
plt.title("Distribution of target over first_active_month")
plt.show()

In [ ]:
train_data['first_active_month'].value_counts()[0:10]

In [ ]:
train_data['first_active_month'].describe()

In [ ]:
plt.figure(figsize = (13,5))
plt.subplot(121)
plt.title('Purchase amount (Historical Transaction)');
train_data['first_active_month'].value_counts().plot(kind='hist');

In [ ]:
test_data['first_active_month'].min(), test_data['first_active_month'].max()

In [ ]:
test_data['first_active_month'] = pd.to_datetime(test_data['first_active_month'],
                                                  format='%Y-%m')

### Feature Engineering
* Aggregations (Historical, New Merchant and Merchant data)
* Feature Transformations (with mappings, data type, mean)
* Feature Engineering with feature creations and subset selections (from 4 to 151)

In [ ]:
print("Earliest purchase date in historical_data :", historical_data['purchase_date'].min())
print("Earliest purchase date in newmerchant_data :", newmerchant_data['purchase_date'].min())
print("Earliest purchase date in historical_data :", historical_data['purchase_date'].max())
print("Earliest purchase date in newmerchant_data :", newmerchant_data['purchase_date'].max())

In [ ]:
sample_date_str = newmerchant_data['purchase_date'][0] # '2018-03-11 14:57:36'
pd.to_datetime(sample_date_str) # Timestamp('2018-03-11 14:57:36')
datetime.datetime.today() - pd.to_datetime(sample_date_str) # Timedelta('1297 days 12:57:01.779383')
sample_date_diff_df = (datetime.datetime.today() - pd.to_datetime(newmerchant_data['purchase_date'])).dt.days  # dt for entire dataframe
sample_date_diff_df[0]

In [ ]:
pd.to_datetime(newmerchant_data['purchase_date'].max()) - pd.to_datetime(sample_date_str)

In [19]:
for df in [historical_data, newmerchant_data]:
    df_purchase_date_max = pd.to_datetime(df['purchase_date'].max())
    
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    #https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    
    # 7 new columns for historical_data
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    
    # Replace "datetime.datetime.today()" and apply floor division
    df['month_diff'] = ((df_purchase_date_max - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    
historical_data['purchase_date'].head(5)

In [ ]:
# 7 new columns for historical_data and newmerchant_data
historical_data.shape, newmerchant_data.shape, train_data.shape, test_data.shape

In [20]:
aggs = {}

# 'dayofweek', 'year'
for col in ['month','hour','weekofyear', 'subsector_id','merchant_id','merchant_category_id']:
            # 'state_id', 'city_id']: # added more features
    aggs[col] = ['nunique']

aggs['purchase_amount'] = ['sum', 'min', 'max', 'mean', 'var', 'median'] # ['sum','max','min','mean','var']
aggs['installments'] = ['sum', 'max', 'mean', 'var'] # 'median', 'min'
aggs['purchase_date'] = ['min', 'max'] # peak-to-peak (maximum - minimum)
aggs['month_lag'] = ['min', 'mean', 'var'] # ['max','min','mean','var']
aggs['month_diff'] = ['mean']
aggs['authorized_flag'] = ['sum', 'mean']
aggs['weekend'] = ['sum', 'mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

# added more features with median for 'installments' and ptp for 'purchase_date'
# aggs['month'] = [ 'min', 'max', 'mean', 'var']

# Group based on 'purchase_amount'
for col in ['category_2','category_3']:
    historical_data[col+'_mean'] = historical_data.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean']    

# Eg: 'hist_month_nunique', 'hist_hour_nunique'
new_columns = create_new_columns('hist',aggs)
print("New columns :\n", new_columns)
df_hist_trans_group = historical_data.groupby('card_id').agg(aggs)

df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['hist_purchase_date_diff'] = (df_hist_trans_group['hist_purchase_date_max'] - df_hist_trans_group['hist_purchase_date_min']).dt.days
df_hist_trans_group['hist_purchase_date_average'] = df_hist_trans_group['hist_purchase_date_diff']/df_hist_trans_group['hist_card_id_size']
df_hist_trans_group['hist_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['hist_purchase_date_max']).dt.days

print()
print("Shape of df_hist_trans_group :\n", df_hist_trans_group.shape)
print("df_hist_trans_group columns :\n", df_hist_trans_group.columns)

df_train = train_data.merge(df_hist_trans_group, on='card_id',how='left')
df_test = test_data.merge(df_hist_trans_group, on='card_id',how='left')

In [ ]:
# df_train['hist_category_2_mean_mean'].head(5)
# historical_data['purchase_amount'].head(5)
# check_missing_values(df_hist_trans_group)

In [21]:
# After merging with historical data with 'left' with group by card_id
print(df_hist_trans_group.shape, df_train.shape, df_test.shape)

# Release memory
del df_hist_trans_group # train_data, test_data
gc.collect()

In [22]:
# Check for missing values
# train_data['new_hist_first_buy'].isna().any()
check_missing_values(df_train), check_missing_values(df_test)

In [23]:
# ((201917, 50), (123623, 49))
train_data.shape, test_data.shape, df_train.shape, df_test.shape, historical_data.shape, newmerchant_data.shape

#### Exploring New Merchant data

In [ ]:
# There are no duplicate card_ids in train_data with 201,917 transactions
train_data_cards = train_data['card_id']
train_data_cards.nunique()

In [ ]:
test_data_cards = test_data['card_id']
# There are no duplicate card_ids in train_data with 123,623 transactions
test_data_cards.nunique()

In [ ]:
print("Total number of unique cards in train_data and test_data :", (train_data_cards.nunique() + test_data_cards.nunique()))

historical_data_cards = historical_data['card_id']
# There are no duplicate card_ids in train_data with 123,623 transactions
historical_data_cards.nunique()

#### Note :
> There are NO intersected values between train and test sets

> All unique card_ids in train_data and test_data can be found in historical_data

> All unique card_ids in train_data and test_data can be found in newmerchant_data

In [ ]:
# Total number of unique cards in train_data and test_data = Total of unique cards in historical_data
intersect_list = intersection(train_data_cards, test_data_cards)
print("There are NO intersected card_id :", intersect_list)

# np.intersect1d(train_data_cards, test_data_cards)
intersect_list = list(set(train_data_cards) & set(test_data_cards))
print("No. of intersected card_ids :", len(intersect_list))

In [ ]:
#  pandas.core.series.Series
# s1 = pd.Series([4, 5, 20, 42, 42, 43])
# s2 = pd.Series([1, 2, 3, 5, 42])
# set(s1)&set(s2) same results
# np.intersect1d(s1, s2)

In [ ]:
# Difference in card_ids
card_difference = set(train_data_cards).symmetric_difference(test_data_cards)
print("Total no. of card_ids in train_data and test_data which are unique : {}".format(len(card_difference)))

In [ ]:
union_cards_df = pd.concat([train_data_cards, test_data_cards])

# type(union_cards_df) is pandas.core.series.Series
# union_data_size = train_data_cards.nunique() + test_data_cards.nunique() = 325,540
# The card_ids in historical_data intersects with all the card_ids found in train and test sets
# intersect_list = intersection(historical_data_cards, union_cards_df)
# len(intersect_list)

card_difference = set(historical_data_cards).symmetric_difference(union_cards_df)
print("The card_ids in train and test but NOT found in historical_data : {}".format(len(card_difference)))

In [ ]:
# union_data_size = train_data_cards.nunique() + test_data_cards.nunique() = 325,540
# Union returns a new set with elements from the set and the specified iterables
card_union = set(train_data_cards).union(test_data_cards)
print("Total no. of unique cards : ", len(card_union))

# symmetric_difference returns a new set with elements in either the set or the specified iterable but not both.
# The card_ids in historical_data can be found in train and test sets; there are no difference in card_ids
# card_difference = set(historical_data_cards).symmetric_difference(union_cards_df)
card_difference = card_union.symmetric_difference(historical_data_cards)
print("Unique cards of train and test data but not in historical data : ", len(card_difference))

In [ ]:
newmerchant_data_cards = newmerchant_data['card_id']
# There are 290,001 unique cards in newmerchant_data but train_data has 201,917 cards
print("No. of unique cards in train_data :", train_data_cards.nunique())
print("No. of unique cards in newmerchant_data :", newmerchant_data_cards.nunique())

intersect_list_train_newmerchant = intersection(train_data_cards, newmerchant_data_cards)
print("No. of intersected card_ids between train_data_cards and newmerchant_data_cards :", len(intersect_list_train_newmerchant))

# These cards create NaNs during merging
print("Unique card_ids in train_data but not in the newmerchant_data : ",
      train_data_cards.nunique() - len(intersect_list_train_newmerchant))

In [ ]:
card_difference = set(train_data_cards).symmetric_difference(newmerchant_data_cards)
print("Unique card_ids not in the intersection of train_data and the newmerchant_data : ", len(card_difference))

In [ ]:
# There are 290,001 unique cards in newmerchant_data but train_data has 201,917 cards
print("No. of unique cards in test_data :", test_data_cards.nunique())
print("No. of unique cards in newmerchant_data :", newmerchant_data_cards.nunique())

intersect_list_test_newmerchant = intersection(test_data_cards, newmerchant_data_cards)
print("No. of intersected card_ids between test_data_cards and newmerchant_data_cards :", len(intersect_list_test_newmerchant))

# These cards create NaNs during merging
print("Unique card_ids in test_data but not in the newmerchant_data : ",
      test_data_cards.nunique() - len(intersect_list_test_newmerchant))

In [ ]:
train_test_newmerchant_card_union = set(intersect_list_train_newmerchant).union(intersect_list_test_newmerchant)
# len(intersect_list_train_newmerchant) + len(intersect_list_test_newmerchant) gives same result of 290,001
len(train_test_newmerchant_card_union)

In [ ]:
card_difference = train_test_newmerchant_card_union.symmetric_difference(newmerchant_data_cards)
print("Verify that intersected unique cards of train and test data with newmerchant data but not in newmerchant data : ", len(card_difference))

#### Exploring Merchant data

In [ ]:
merchants_data_ids = merchants_data['merchant_id']
newmerchants_data_ids = newmerchant_data['merchant_id']

# There are 334,633 merchant_id in merchants_data but newmerchant_data has only 226,129 merchant_id
print("No. of unique merchant_id in merchants_data :", merchants_data_ids.nunique())
print("No. of unique merchant_id in newmerchant_data :", newmerchants_data_ids.nunique())

intersect_list_newmerchants_merchants = intersection(newmerchants_data_ids, merchants_data_ids)
print("No. of intersected merchant_ids between newmerchants_data_ids and merchants_data_ids :", len(intersect_list_newmerchants_merchants))

# All merchant_id in newmerchant_data can be found in merchants_data
print("Unique merchant_ids in newmerchants_data but not in the merchants_data :",
      newmerchants_data_ids.nunique() - len(intersect_list_newmerchants_merchants))

In [ ]:
merchants_data_cat_ids = merchants_data['merchant_category_id']
newmerchants_data_cat_ids = newmerchant_data['merchant_category_id']

# There are 334,633 merchant_id in merchants_data but newmerchant_data has only 226,129 merchant_id
print("No. of unique merchant_category_id in merchants_data :", merchants_data_cat_ids.nunique())
print("No. of unique merchant_category_id in newmerchant_data :", newmerchants_data_cat_ids.nunique())

intersect_list_newmerchants_merchants_cat_id = intersection(newmerchants_data_cat_ids, merchants_data_cat_ids)
print("No. of intersected merchant_category_ids between newmerchants_data_cat_ids and merchants_data_cat_ids :",
      len(intersect_list_newmerchants_merchants_cat_id))

# These merchant_category_id create NaNs during merging
print("Unique merchant_category_ids in newmerchants_data but not in the merchants_data :",
      newmerchants_data_cat_ids.nunique() - len(intersect_list_newmerchants_merchants_cat_id))

In [ ]:
merchants_data_ids = merchants_data['merchant_id']
historical_data_merchant_ids = historical_data['merchant_id']

# There are 334,633 merchant_id in merchants_data but newmerchant_data has only 226,129 merchant_id
print("No. of unique merchant_id in merchants_data :", merchants_data_ids.nunique())
print("No. of unique merchant_id in historical_data :", historical_data_merchant_ids.nunique())

intersect_list_historical_merchants = intersection(historical_data_merchant_ids, merchants_data_ids)
print("No. of intersected merchant_ids between historical_data_merchant_ids and merchants_data_ids :",
      len(intersect_list_historical_merchants))

# All merchant_id in historical_data can be found in merchants_data
print("Unique merchant_ids in historical_data but not in the merchants_data :",
      historical_data_merchant_ids.nunique() - len(intersect_list_historical_merchants))

In [ ]:
merchants_data_cat_ids = merchants_data['merchant_category_id']
historical_data_cat_ids = historical_data['merchant_category_id']

# There are 334,633 merchant_id in merchants_data but newmerchant_data has only 226,129 merchant_id
print("No. of unique merchant_category_id in merchants_data :", merchants_data_cat_ids.nunique())
print("No. of unique merchant_category_id in historical_data :", historical_data_cat_ids.nunique())

intersect_list_historical_merchants_cat_id = intersection(historical_data_cat_ids, merchants_data_cat_ids)
print("No. of intersected merchant_category_ids between historical_data_cat_ids and merchants_data_cat_ids :",
      len(intersect_list_historical_merchants_cat_id))

# These merchant_category_id create NaNs during merging
print("Unique merchant_category_ids in historical_data but not in the merchants_data :",
      historical_data_cat_ids.nunique() - len(intersect_list_historical_merchants_cat_id))

print()
print("There are MORE merchant_category_ids in historical_data than in the merchants_data :",
      (historical_data_cat_ids.nunique() - merchants_data_cat_ids.nunique()))

In [ ]:
merchants_data['merchant_id'].describe()

In [24]:
df_train.shape, df_test.shape, newmerchant_data.shape, merchants_data.shape

In [ ]:
check_missing_values(newmerchant_data), check_missing_values(merchants_data)

In [ ]:
# There are duplicate values of 'merchant_id'
merchants_data['merchant_id'].describe()

## Impute missing values in merchants data with respective mode values

In [ ]:
print(merchants_data['avg_sales_lag3'].isna().value_counts())
print()
print("Mode value of avg_sales_lag3 :", merchants_data['avg_sales_lag3'].mode()[0])

missing_avg_sales_l3_index = merchants_data.loc[merchants_data['avg_sales_lag3'].isna(), 'merchant_id'].index.values
print("The index of missing avg_sales_lag3 in merchants_data :", missing_avg_sales_l3_index)

merchants_data.loc[merchants_data['avg_sales_lag3'].isna()]

In [25]:
# Fill missing values in merchants_data with respective mode values
merchants_data['avg_sales_lag3'].fillna(merchants_data['avg_sales_lag3'].mode()[0], inplace = True)

In [ ]:
print(merchants_data['avg_sales_lag6'].isna().value_counts())
print()
print("Mode value of avg_sales_lag6 :", merchants_data['avg_sales_lag6'].mode()[0])

missing_avg_sales_l6_index = merchants_data.loc[merchants_data['avg_sales_lag6'].isna(), 'merchant_id'].index.values
print("The index of missing avg_sales_lag6 in merchants_data :", missing_avg_sales_l6_index)

merchants_data.loc[merchants_data['avg_sales_lag6'].isna()]

In [26]:
merchants_data['avg_sales_lag6'].fillna(merchants_data['avg_sales_lag6'].mode()[0], inplace = True)

In [ ]:
print(merchants_data['avg_sales_lag12'].isna().value_counts())
print()
print("Mode value of avg_sales_lag12 :", merchants_data['avg_sales_lag12'].mode()[0])

missing_avg_sales_l12_index = merchants_data.loc[merchants_data['avg_sales_lag12'].isna(), 'merchant_id'].index.values
print("The index of missing avg_sales_lag12 in merchants_data :", missing_avg_sales_l12_index)

merchants_data.loc[merchants_data['avg_sales_lag12'].isna()]

In [27]:
merchants_data['avg_sales_lag12'].fillna(merchants_data['avg_sales_lag12'].mode()[0], inplace = True)

In [ ]:
print(merchants_data['category_2'].isna().value_counts())
print()
print("Mode value of category_2 :", merchants_data['category_2'].mode()[0])

missing_category_2_index = merchants_data.loc[merchants_data['category_2'].isna(), 'merchant_id'].index.values
print("The index of missing category_2 in merchants_data :", missing_category_2_index)

merchants_data.loc[missing_category_2_index]

In [28]:
merchants_data['category_2'].fillna(merchants_data['category_2'].mode()[0], inplace = True)
merchants_data.loc[3:15]

In [29]:
check_missing_values(merchants_data), merchants_data.shape, newmerchant_data.shape

#### Merging strategy :
> 1. Aggregate historical data with ***'card_id'*** and merge to train and test with groupby ***'card_id'***
> 2. Aggregate merchant data with ***'merchant_id'*** and merge to new merchant data with groupby ***'merchant_id'***
> 3. Merge new merchant data with merchant data to new train and test with groupby ***'card_id'***

In [ ]:
# merchants_data['most_recent_sales_range'].value_counts()

# merchants_data['most_recent_purchases_range'].value_counts()
# merchants_data['most_recent_purchases_range'].min(), merchants_data['most_recent_purchases_range'].max()

# merchants_data['most_recent_sales_range'].mode()

# aggs = {}
# aggs['most_recent_sales_range'] = ['min']
# md_tmp = merchants_data.groupby('merchant_id').agg(aggs)
# md_tmp.shape

# md_tmp = pd.DataFrame()
# Group based on 'purchase_amount'
# for col in ['most_recent_sales_range','most_recent_purchases_range']:
#     md_tmp[col+'_avg_sales_lag3_mean'] = merchants_data.groupby([col])['avg_sales_lag3'].transform('min')
#     md_tmp[col+'_avg_sales_lag6_mean'] = merchants_data.groupby([col])['avg_sales_lag6'].transform('min')
#     md_tmp[col+'_avg_sales_lag12_mean'] = merchants_data.groupby([col])['avg_sales_lag12'].transform('min')

# md_tmp.reset_index(drop=False,inplace=True)
# md_tmp.head(2)
# check_missing_values(md_tmp)

#### Feature mappings

In [31]:
for df in [merchants_data]:
    df['most_recent_sales_range'] = df['most_recent_sales_range'].map({'A':5, 'B':4, 'C':3, 'D':2, 'E':1})
    df['most_recent_purchases_range'] = df['most_recent_purchases_range'].map({'A':5, 'B':4, 'C':3, 'D':2, 'E':1})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0})

In [32]:
# Majority revenue and quantity of transactions in last active month are high in merchants data
df['most_recent_sales_range'].value_counts(), df['most_recent_purchases_range'].value_counts(), df['category_1'].value_counts()

In [35]:
aggs = {}

for col in ['merchant_group_id','subsector_id','category_1','category_4','city_id', 'state_id']: # 'merchant_id'
    aggs[col] = ['nunique']

aggs['numerical_1'] = ['sum', 'mean']
aggs['numerical_2'] = ['sum', 'mean']
aggs['avg_sales_lag3'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['avg_purchases_lag3'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['active_months_lag3'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['avg_sales_lag6'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['avg_purchases_lag6'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['active_months_lag6'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['avg_sales_lag12'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['avg_purchases_lag12'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['active_months_lag12'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['category_2'] = ['mean']
# Explore more features
aggs['most_recent_sales_range'] = ['min', 'max'] 
aggs['most_recent_purchases_range'] = ['min', 'max']

aggs['category_1'] = ['min', 'max']

new_columns = create_new_columns('merchants',aggs)
print("New columns :\n", new_columns)
df_merchant_id_group = merchants_data.groupby('merchant_id').agg(aggs) # merchant_category_id

df_merchant_id_group.columns = new_columns
df_merchant_id_group.reset_index(drop=False,inplace=True)

print("Shape of df_merchant_category :\n", df_merchant_id_group.shape)
print("df_merchant_id_group columns :\n", df_merchant_id_group.columns)
df_merchant_id_group

In [36]:
# Group newmerchant_data and merchants_data
newmerchant_with_merchants_data = newmerchant_data.merge(df_merchant_id_group, on='merchant_id',how='left') # merchant_category_id
newmerchant_with_merchants_data.shape

In [37]:
df_train.shape, df_test.shape, newmerchant_data.shape, merchants_data.shape, df_merchant_id_group.shape

#### When merging with 'merchant_category_id', there are infinite values that should be replaced

In [ ]:
# np.isinf(df_merchant_id_group).any()
# np.isinf(df_merchant_id_group).any().value_counts()

In [ ]:
# print("printing column name where infinity is present")
# col_name = df_merchant_id_group.columns.to_series()[np.isinf(df_merchant_id_group).any()]

# print(col_name)

In [ ]:
# print(df_merchant_id_group['merchants_avg_purchases_lag3_sum'].median())
# print(df_merchant_id_group['merchants_avg_purchases_lag3_mean'].median())
# print(df_merchant_id_group['merchants_avg_purchases_lag6_sum'].median())
# print(df_merchant_id_group['merchants_avg_purchases_lag6_mean'].median())
# print(df_merchant_id_group['merchants_avg_purchases_lag12_sum'].median())
# print(df_merchant_id_group['merchants_avg_purchases_lag12_mean'].median())

# missing_value_index = df_merchant_id_group.loc[np.isinf(df_merchant_id_group['merchants_avg_purchases_lag3_sum']),
#                                                'merchants_avg_purchases_lag3_sum'].index.values

# missing_value_index

# df_merchant_id_group.loc[missing_value_index, 'merchants_avg_purchases_lag3_sum']

In [ ]:
# Replace all infinite values to NaN which will replaced with median values collectively
# df_merchant_id_group.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
# merchants_data['merchant_id'].describe()

# newmerchant_with_merchants_data.columns

# merchants_data['merchant_category_id'].nunique(), newmerchant_data['merchant_category_id'].nunique()

# newmerchant_with_merchants_data.head(2)

In [38]:
check_missing_values(newmerchant_with_merchants_data), newmerchant_with_merchants_data.shape

In [ ]:
# # Define the aggregation procedure outside of the groupby operation
# aggregations = {
#     'purchase_amount': ['sum', 'mean', 'std', 'min', 'max', 'size', 'median']
# }

# grouped = newmerchant_data.groupby('card_id').agg(aggregations)
# grouped.columns = grouped.columns.droplevel(level=0)
# grouped.rename(columns={
#     "sum": "sum_purchase_amount", 
#     "mean": "mean_purchase_amount",
#     "std": "std_purchase_amount", 
#     "min": "min_purchase_amount",
#     "max": "max_purchase_amount", 
#     "size": "num_purchase_amount",
#     "median": "median_purchase_amount"
# }, inplace=True)
# grouped.reset_index(inplace=True)

# grouped.shape

# df_train = pd.merge(df_train, grouped, on="card_id", how="left")
# df_test = pd.merge(df_test, grouped, on="card_id", how="left")

# del grouped
# gc.collect()

In [ ]:
# Due to card_ids not found in newmerchant_data
# check_missing_values(df_train), check_missing_values(df_test)

#### Merging with new merchant data

In [39]:
aggs = {}

# 'year', 'month', 'weekofyear', 'dayofweek', 'hour', 'merchant_id', 'subsector_id',
for col in ['merchant_category_id']:
            # 'city_id', 'state_id']: # added more features
    aggs[col] = ['nunique']

aggs['purchase_amount'] = ['sum', 'min', 'max', 'mean', 'median'] # ['sum','max','min','mean','var'], 'std'
aggs['installments'] = ['sum', 'max'] # ['sum','max','min','mean','var'], ['std', 'median']
aggs['purchase_date'] = ['min', 'max']
aggs['month_lag'] = ['mean'] # ['max','min','mean','var'], 'std'
aggs['month_diff'] = ['mean']
aggs['weekend'] = ['mean'] # 'sum', 
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

# include columns with merchant information
# aggs['merchants_merchant_group_id_nunique'] = ['size']
# aggs['merchants_subsector_id_nunique'] = ['size']
# aggs['merchants_category_1_nunique'] = ['size']
# aggs['merchants_most_recent_sales_range_nunique'] = ['size']
# aggs['merchants_most_recent_purchases_range_nunique'] = ['size']
# aggs['merchants_category_4_nunique'] = ['size']
# aggs['merchants_city_id_nunique'] = ['size']
# aggs['merchants_state_id_nunique'] = ['size']

# 'new_merchant_merchants_active_months_lag3_mean_max',
# 'new_merchant_merchants_active_months_lag3_mean_min',
# 'new_merchant_merchants_active_months_lag3_mean_sum',

# 'new_merchant_merchants_active_months_lag3_sum_min',
# 'new_merchant_merchants_active_months_lag3_sum_max',

# 'new_merchant_merchants_active_months_lag6_sum_min',
# 'new_merchant_merchants_active_months_lag6_sum_max',
# 'new_merchant_merchants_active_months_lag6_mean_min',
# 'new_merchant_merchants_active_months_lag6_mean_max',

# 'new_merchant_merchants_active_months_lag12_sum_min',
# 'new_merchant_merchants_active_months_lag12_sum_max',
# 'new_merchant_merchants_active_months_lag12_mean_max',

aggs['merchants_numerical_1_mean'] = ['min', 'max']
aggs['merchants_numerical_2_mean'] = ['min', 'max']

# aggs['avg_sales_lag3'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['merchants_avg_sales_lag3_sum'] = ['min', 'max', 'mean']
aggs['merchants_avg_sales_lag3_mean'] = ['min', 'max']
# aggs['merchants_avg_sales_lag3_size'] = ['min', 'max']
aggs['merchants_avg_sales_lag3_min'] = ['min', 'max']
aggs['merchants_avg_sales_lag3_max'] = ['min', 'max']

# aggs['avg_purchases_lag3'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['merchants_avg_purchases_lag3_sum'] = ['min', 'max', 'mean']
aggs['merchants_avg_purchases_lag3_mean'] = ['min', 'max']
# aggs['merchants_avg_purchases_lag3_size'] = ['min', 'max']
aggs['merchants_avg_purchases_lag3_min'] = ['min', 'max']
aggs['merchants_avg_purchases_lag3_max'] = ['min', 'max']

## aggs['active_months_lag3'] = ['sum', 'mean', 'size', 'min', 'max']
# aggs['merchants_active_months_lag3_sum'] = ['mean'] # 'min', 'max',
# aggs['merchants_active_months_lag3_mean'] = ['mean', 'median'] # 'sum', 'min', 'max'
# aggs['merchants_active_months_lag3_size'] = ['min', 'max']
# aggs['merchants_active_months_lag3_min'] = ['min', 'max']
# aggs['merchants_active_months_lag3_max'] = ['min', 'max']

# aggs['avg_sales_lag6'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['merchants_avg_sales_lag6_sum'] = ['min', 'max', 'mean']
aggs['merchants_avg_sales_lag6_mean'] = ['min', 'max']
# aggs['merchants_avg_sales_lag6_size'] = ['min', 'max']
aggs['merchants_avg_sales_lag6_min'] = ['min', 'max']
aggs['merchants_avg_sales_lag6_max'] = ['min', 'max']

# aggs['avg_purchases_lag6'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['merchants_avg_purchases_lag6_sum'] = ['min', 'max', 'mean']
aggs['merchants_avg_purchases_lag6_mean'] = ['min'] # 'max'
# aggs['merchants_avg_purchases_lag6_size'] = ['min', 'max']
aggs['merchants_avg_purchases_lag6_min'] = ['min'] # 'max'
aggs['merchants_avg_purchases_lag6_max'] = ['min', 'max']

# aggs['active_months_lag6'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['merchants_active_months_lag6_sum'] = ['mean'] # 'min', 'max'
# aggs['merchants_active_months_lag6_mean'] = ['min', 'max']
# aggs['merchants_active_months_lag6_size'] = ['min', 'max']
# aggs['merchants_active_months_lag6_min'] = ['min', 'max']
# aggs['merchants_active_months_lag6_max'] = ['min', 'max']

# aggs['avg_sales_lag12'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['merchants_avg_sales_lag12_sum'] = ['min', 'max', 'mean']
aggs['merchants_avg_sales_lag12_mean'] = ['min', 'max']
# aggs['merchants_avg_sales_lag12_size'] = ['min', 'max']
aggs['merchants_avg_sales_lag12_min'] = ['min', 'max']
aggs['merchants_avg_sales_lag12_max'] = ['min', 'max']

# aggs['avg_purchases_lag12'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['merchants_avg_purchases_lag12_sum'] = ['min', 'max', 'mean']
aggs['merchants_avg_purchases_lag12_mean'] = ['min', 'max']
# aggs['merchants_avg_purchases_lag12_size'] = ['min', 'max'] 
aggs['merchants_avg_purchases_lag12_min'] = ['min', 'max']
aggs['merchants_avg_purchases_lag12_max'] = ['min', 'max']

# aggs['active_months_lag12'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['merchants_active_months_lag12_sum'] = ['min', 'mean'] #  'max'
aggs['merchants_active_months_lag12_mean'] = ['min', 'max']
# aggs['merchants_active_months_lag12_size'] = ['min', 'max']
aggs['merchants_active_months_lag12_min'] = ['min', 'max']
# aggs['merchants_active_months_lag12_max'] = ['min', 'max']

aggs['merchants_most_recent_sales_range_min'] = ['min', 'max']
aggs['merchants_most_recent_sales_range_max'] = ['min', 'max']
aggs['merchants_most_recent_purchases_range_min'] = ['min', 'max']
aggs['merchants_most_recent_purchases_range_max'] = ['min', 'max']

aggs['merchants_category_1_min'] = ['min', 'max']
aggs['merchants_category_1_max'] = ['min', 'max']

## To be continue for further feature explorations
##

aggs['merchants_numerical_1_sum'] = ['min', 'max', 'mean']
aggs['merchants_numerical_2_sum'] = ['min', 'max', 'mean']

aggs['merchants_avg_purchases_lag6_sum'] = ['min', 'max', 'mean']

# aggs['merchants_active_months_lag6_sum'] = ['mean'] # 'min', 'max'
# aggs['merchants_active_months_lag6_mean'] = ['sum'] #  'min', 'max'

aggs['merchants_avg_sales_lag12_sum'] = ['sum', 'min', 'max', 'mean']
aggs['merchants_avg_purchases_lag12_sum'] = ['min', 'max', 'mean']

# aggs['merchants_active_months_lag12_sum'] = ['mean'] # 'min', 'max', 
aggs['merchants_active_months_lag12_mean'] = ['sum'] # , 'min', 'max'

aggs['merchants_category_2_mean'] = ['sum'] # , 'min', 'max'

# added more features with median for 'installments'
# aggs['month'] = [ 'min', 'max', 'mean', 'var']

for col in ['category_2','category_3']:
    # Replace newmerchant_data with newmerchant_with_merchants_data
    newmerchant_with_merchants_data[col+'_mean'] = newmerchant_with_merchants_data.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean']
    
new_columns = create_new_columns('new_merchant',aggs)
print("New columns :\n", new_columns)
df_new_merchant_trans_group = newmerchant_with_merchants_data.groupby('card_id').agg(aggs) # newmerchant_data

df_new_merchant_trans_group.columns = new_columns
df_new_merchant_trans_group.reset_index(drop=False,inplace=True)
df_new_merchant_trans_group['new_merchant_purchase_date_diff'] = (df_new_merchant_trans_group['new_merchant_purchase_date_max'] - df_new_merchant_trans_group['new_merchant_purchase_date_min']).dt.days
df_new_merchant_trans_group['new_merchant_purchase_date_average'] = df_new_merchant_trans_group['new_merchant_purchase_date_diff'] / df_new_merchant_trans_group['new_merchant_card_id_size']
df_new_merchant_trans_group['new_merchant_purchase_date_uptonow'] = (datetime.datetime.today() - df_new_merchant_trans_group['new_merchant_purchase_date_max']).dt.days

print()
print("Shape of df_new_merchant_trans_group :\n", df_new_merchant_trans_group.shape)

In [ ]:
df_new_merchant_trans_group['new_merchant_purchase_date_diff']

In [40]:
df_train.shape, df_test.shape

In [41]:
# Get ready for training models
X_train = df_train.merge(df_new_merchant_trans_group,on='card_id',how='left')
X_test = df_test.merge(df_new_merchant_trans_group,on='card_id',how='left')

In [42]:
# After merging with historical data with 'left' with group by card_id
print(df_new_merchant_trans_group.shape, X_train.shape, X_test.shape) # ((201917, 91), (123623, 90))
del df_new_merchant_trans_group #, train_data, test_data
gc.collect()

In [43]:
X_train['hist_purchase_date_max'].head(2), X_test['hist_purchase_date_max'].head(2)

In [44]:
X_train['new_merchant_purchase_date_max'].head(2), X_test['new_merchant_purchase_date_max'].head(2)

In [45]:
X_train['hist_purchase_amount_max'].head(2), X_test['hist_purchase_amount_max'].head(2)

In [46]:
X_train['new_merchant_purchase_amount_max'].head(2), X_test['new_merchant_purchase_amount_max'].head(2)

In [ ]:
X_train.columns

In [ ]:
# df_train = df_train.merge(df_merchant_category, on='merchant_category_id', how='left')
# df_test = df_test.merge(df_merchant_category, on='merchant_category_id', how='left')

# df_train.shape, df_test.shape
# check_missing_values(X_train), check_missing_values(X_test)

#### Concatenate the categorical features

In [ ]:
# Numerical representations of the nominal feature_1, feature_2, feature_3
X_train = pd.concat([X_train, ohe_train_df_1, ohe_train_df_2, ohe_train_df_3], axis=1, sort=False)
X_test = pd.concat([X_test, ohe_test_df_1, ohe_test_df_2, ohe_test_df_3], axis=1, sort=False)

del ohe_train_df_1, ohe_train_df_2, ohe_train_df_3
del ohe_test_df_1, ohe_test_df_2, ohe_test_df_3
gc.collect()

In [48]:
# (201917, 101), (123623, 100)
print(X_train.shape, X_test.shape)

In [ ]:
loyality_score = X_train['target']
ax = loyality_score.plot.hist(bins=20, figsize=(6, 5))
_ = ax.set_title("target histogram")
plt.show()

fig, axs = plt.subplots(1,2, figsize=(12, 5))
_ = loyality_score[loyality_score > 10].plot.hist(ax=axs[0])
_ = axs[0].set_title("target histogram for values greater than 10")
_ = loyality_score[loyality_score < -10].plot.hist(ax=axs[1])
_ = axs[1].set_title("target histogram for values less than -10")
plt.show()

In [ ]:
# There are 2207 rows with target values less than -30
X_train.loc[X_train['target'] < -30, 'target'].value_counts()

In [ ]:
X_train['outliers'] = 0

# Consider the target values less ta than -30 are outliers
X_train.loc[X_train['target'] < -30, 'outliers'] = 1
X_train['outliers'].value_counts()

In [ ]:
for f in ['feature_1','feature_2','feature_3']:
    # Setting mean value of the 'outliers' for the input features 
    order_label = X_train.groupby([f])['outliers'].mean()
    X_train[f] = X_train[f].map(order_label)
    X_test[f] = X_test[f].map(order_label)
    
X_train.shape, X_test.shape

In [ ]:
X_train.loc[0:5, ['feature_1','feature_2','feature_3']]

In [ ]:
print(X_train.columns)

In [ ]:
X_train['hist_purchase_date_min'][0]

In [ ]:
# X_train[['hist_purchase_date_min', 'hist_purchase_date_max', 'new_merchant_purchase_date_min', 'new_merchant_purchase_date_max']]
# X_train[['hist_purchase_date_min', 'hist_purchase_date_max', 'new_merchant_purchase_date_min', 'new_merchant_purchase_date_max']].isna().any()
# X_test[['hist_purchase_date_min', 'hist_purchase_date_max', 'new_merchant_purchase_date_min', 'new_merchant_purchase_date_max']].isna().any()

print("Missing purchase date (min-max) in X_train : ")
print("min mode : {}\nmax mode : {}".format(X_train['new_merchant_purchase_date_min'].mode(),
                                            X_train['new_merchant_purchase_date_max'].mode()))
print("-" * 80)
print("Missing purchase date (min-max) in X_test : ")
print("min mode : {}\nmax mode : {}".format(X_test['new_merchant_purchase_date_min'].mode(),
                                            X_test['new_merchant_purchase_date_max'].mode()))

In [ ]:
type(X_train['new_merchant_purchase_date_min'].mode()[0]), type(X_train['new_merchant_purchase_date_max'].mode()[0])

In [ ]:
print("Purchase data in historical_data")
print(historical_data['purchase_date'].min(), historical_data['purchase_date'].max())
print("\nPurchase data in newmerchant_data")
print(newmerchant_data['purchase_date'].min(), newmerchant_data['purchase_date'].max())

#### Impute missing purchase min-max values in train and test data

In [53]:
# Fill missing values for min-max purchase date newmerchant_data from after merging with newmmerchant_data for the card_ids in train_data
X_train['new_merchant_purchase_date_min'].fillna(newmerchant_data['purchase_date'].min(), inplace = True)
X_train['new_merchant_purchase_date_max'].fillna(newmerchant_data['purchase_date'].max(), inplace = True)
X_train['new_merchant_purchase_amount_min'].fillna(newmerchant_data['purchase_amount'].min(), inplace = True)
X_train['new_merchant_purchase_amount_max'].fillna(newmerchant_data['purchase_amount'].max(), inplace = True)

X_test['new_merchant_purchase_date_min'].fillna(newmerchant_data['purchase_date'].min(), inplace = True)
X_test['new_merchant_purchase_date_max'].fillna(newmerchant_data['purchase_date'].max(), inplace = True)
X_test['new_merchant_purchase_amount_min'].fillna(newmerchant_data['purchase_amount'].min(), inplace = True)
X_test['new_merchant_purchase_amount_max'].fillna(newmerchant_data['purchase_amount'].max(), inplace = True)

In [54]:
X_train[['hist_purchase_date_min', 'hist_purchase_date_max', 'new_merchant_purchase_date_min', 'new_merchant_purchase_date_max',
         'new_merchant_purchase_amount_min', 'new_merchant_purchase_amount_max']].info()

In [55]:
X_test[['hist_purchase_date_min', 'hist_purchase_date_max', 'new_merchant_purchase_date_min', 'new_merchant_purchase_date_max',
        'new_merchant_purchase_amount_min', 'new_merchant_purchase_amount_max']].info()

In [56]:
# Convert Timestamp to int64 with imputed missing values
for f in ['hist_purchase_date_min', 'hist_purchase_date_max', 'new_merchant_purchase_date_min', 'new_merchant_purchase_date_max']:
    print(type(X_train[f][0]))
    print(type(X_test[f][0]))
    X_train[f] = X_train[f].astype(np.int64) * 1e-9
    X_test[f] = X_test[f].astype(np.int64) * 1e-9

In [57]:
X_train.shape, X_test.shape

In [58]:
# This strong ration feature elevates the ranking by 10%+
X_train['new_hist_purchase_date_max_ratio'] = X_train['new_merchant_purchase_date_max'] / X_train['hist_purchase_date_max']
X_test['new_hist_purchase_date_max_ratio'] = X_test['new_merchant_purchase_date_max'] / X_test['hist_purchase_date_max']

# Need to try if the ratio improves the overall RMSE
X_train['new_hist_purchase_amount_max_ratio'] = X_train['new_merchant_purchase_amount_max'] / X_train['hist_purchase_amount_max']
X_test['new_hist_purchase_amount_max_ratio'] = X_test['new_merchant_purchase_amount_max'] / X_test['hist_purchase_amount_max']

# Need to produce 'card_purchase_date_max_ratio'

X_train.shape, X_test.shape

In [ ]:
# for f in ['new_merchant_purchase_date_max',
#           'hist_purchase_date_max',]:
#     X_train[f +'_int'] = X_train[f].astype(np.int64) * 1e-9
#     X_test[f +'_int'] = X_test[f].astype(np.int64) * 1e-9
    
# X_train['card_purchase_date_max_ratio'] = X_train['new_merchant_purchase_date_max_int'] / X_train['hist_purchase_date_max_int']
# X_test['card_purchase_date_max_ratio'] = X_test['new_merchant_purchase_date_max_int'] / X_test['hist_purchase_date_max_int']

In [61]:
X_train['new_merchant_merchants_category_1_min_min'].fillna(newmerchant_data['category_1'].min(), inplace = True)
X_train['new_merchant_merchants_category_1_min_max'].fillna(newmerchant_data['category_1'].min(), inplace = True)
X_train['new_merchant_merchants_category_1_max_min'].fillna(newmerchant_data['category_1'].max(), inplace = True)
X_train['new_merchant_merchants_category_1_max_max'].fillna(newmerchant_data['category_1'].max(), inplace = True)

X_test['new_merchant_merchants_category_1_min_min'].fillna(newmerchant_data['category_1'].min(), inplace = True)
X_test['new_merchant_merchants_category_1_min_max'].fillna(newmerchant_data['category_1'].min(), inplace = True)
X_test['new_merchant_merchants_category_1_max_min'].fillna(newmerchant_data['category_1'].max(), inplace = True)
X_test['new_merchant_merchants_category_1_max_max'].fillna(newmerchant_data['category_1'].max(), inplace = True)

In [62]:
X_train[['new_merchant_merchants_category_1_min_min', 'new_merchant_merchants_category_1_min_max',
         'new_merchant_merchants_category_1_max_min', 'new_merchant_merchants_category_1_max_max']].info()

In [63]:
# excluded_features = ['first_active_month', 'card_id', 'target', 'date', 'year']
# Excluse non numeric features
# excluded_features = ['first_active_month', 'card_id', 'target', 'hist_purchase_date_min', 'hist_purchase_date_max']
# train_features = [c for c in df_train.columns if c not in excluded_features]

# 'hist_purchase_date_min', 'hist_purchase_date_max', 'new_merchant_purchase_date_min', 'new_merchant_purchase_date_max'
# Consider removal of these 'new_merchant_purchase_date_max', 'hist_purchase_date_max' after getting their ratio
excluded_features = ['first_active_month', 'card_id', 'target', 'outliers']
train_features = [c for c in X_train.columns if c not in excluded_features]

print("Features used for training : ", train_features)

In [64]:
# Filled mean information for the missing cards from newmerchant_data
# Fill missing values with mean values; maybe use median value
for col in train_features:
    for df in [X_train, X_test]:
        if df[col].dtype == "float64":
            df[col] = df[col].fillna(df[col].median()) # mean

In [ ]:
# X_train['new_merchant_merchants_category_1_min_min'].mode()[0]

# # Fill missing values for min-max purchase date newmerchant_data from after merging with newmmerchant_data for the card_ids in train_data
# X_train['new_merchant_merchants_category_1_min_min'].fillna(newmerchant_data['category_1'].min(), inplace = True)
# X_train['new_merchant_merchants_category_1_min_max'].fillna(newmerchant_data['category_1'].min(), inplace = True)
# X_train['new_merchant_merchants_category_1_max_min'].fillna(newmerchant_data['category_1'].max(), inplace = True)
# X_train['new_merchant_merchants_category_1_max_max'].fillna(newmerchant_data['category_1'].max(), inplace = True)

# X_test['new_merchant_merchants_category_1_min_min'].fillna(newmerchant_data['category_1'].min(), inplace = True)
# X_test['new_merchant_merchants_category_1_min_max'].fillna(newmerchant_data['category_1'].min(), inplace = True)
# X_test['new_merchant_merchants_category_1_max_min'].fillna(newmerchant_data['category_1'].max(), inplace = True)
# X_test['new_merchant_merchants_category_1_max_max'].fillna(newmerchant_data['category_1'].max(), inplace = True)

In [65]:
len(check_missing_values(X_train)), len(check_missing_values(X_test))

In [ ]:
X_train.info()

In [ ]:
#Finding Correlation between variables of newmerchant_data features
selected_columns = ['category_2', 'month_lag', 'purchase_amount', 'state_id', 'subsector_id', 'installments']
data_frame = newmerchant_data[selected_columns]

data_frame = data_frame.dropna()

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(data_frame.iloc[:,:].values, i) for i in range(data_frame.shape[1])]
vif["features"] = data_frame.columns
vif

In [ ]:
newmerchant_data['category_3'].value_counts()

In [ ]:
newmerchant_data['authorized_flag'].value_counts()

In [ ]:
newmerchant_data.info()

In [ ]:
Dict = {'A':1,'B':2,'C':3}
# value_counts {'Y':1,'N':0}

selected_columns = ['category_2', 'month_lag', 'purchase_amount', 'state_id',
                    'subsector_id', 'installments', 'authorized_flag', 'month_lag', 'category_3'] 
data_frame = newmerchant_data[selected_columns]
data_frame['category_3'] = data_frame['category_3'].map(Dict)
# data_frame['authorized_flag'] = data_frame['authorized_flag'].map(Dict1)

data_frame = data_frame.dropna()

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(data_frame.iloc[:,:].values, i) for i in range(data_frame.shape[1])]
vif["features"] = data_frame.columns
vif

#### Code snippets

In [ ]:
# Aggregate works only for numeric colums
# df_new_merchant_trans_group = temp_df.groupby('card_id').agg(['mean', 'median'])
# df_new_merchant_trans_group.columns = ['1', '2', '3', '4',
#                                        '5', '6', '7', '8',
#                                        '9', '10', '11', '12',
#                                        '13', '14', '15', '16', '17']
# df_new_merchant_trans_group.reset_index(drop=False,inplace=True)
# df_new_merchant_trans_group

# df_new_merchant_trans_group = temp_df.groupby('card_id')
# df_new_merchant_trans_group.obj.columns

# a = temp_df.groupby('card_id').count()
# a.reset_index(drop=False,inplace=True)

# newmerchant_data.loc[newmerchant_data['card_id']=='C_ID_415bb3a509']

# df_new_merchant_trans_group = merchant_test_df.groupby('card_id').agg(['mean', 'median'])
# df_new_merchant_trans_group.columns = ['1', '2', '3', '4',
#                                        '5', '6', '7', '8',
#                                        '9', '10', '11', '12',
#                                        '13', '14', '15', '16', '17']
# df_new_merchant_trans_group.reset_index(drop=False,inplace=True)
# df_new_merchant_trans_group

In [ ]:
# df_train.iloc[df_train['new_merchant_purchase_amount_sum'].isna().any()]
# missing_card_id = test_data.loc[test_data['first_active_month'].isna(), 'card_id'].reset_index(drop=True)[0]
# # get the historical data for the missing card_id
# card_missing_first_active_month = historical_data.loc[historical_data['card_id'] == missing_card_id]

# print("Card_id : {} with {} transactions.".format(missing_card_id,
#                                                   card_missing_first_active_month.shape[0]))

missing_sum_card_id = X_train.loc[X_train['new_merchant_purchase_amount_sum'].isna(), 'card_id'].reset_index(drop=True)[0]

print("Missing sum for card_id: ", missing_sum_card_id)
# df_train.loc[df_train['new_merchant_purchase_amount_sum'].isna().any(), 'new_merchant_purchase_amount_sum']
card_missing_sum = X_train.loc[X_train['card_id'] == missing_sum_card_id]
card_missing_sum.iloc[:, 28:29]

In [ ]:
# There are card_id in train_data but not in newmerchant_data
newmerchant_data.loc[newmerchant_data['card_id'] == missing_sum_card_id]

In [ ]:
X_train.loc[df_train['card_id'] == missing_sum_card_id]

# train_data.loc[train_data['card_id'] == missing_sum_card_id]
# test_data.loc[test_data['card_id'] == missing_sum_card_id]
# historical_data.loc[historical_data['card_id'] == missing_sum_card_id]

In [ ]:
print(df_new_merchant_trans_group.shape, df_train.shape, df_test.shape)
del df_new_merchant_trans_group; gc.collect()

In [ ]:
# More Feature Engineering
for df in [df_train, df_test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max', 'new_hist_purchase_date_min']:
        df[f] = df[f].fillna(datetime.datetime.today(),inplace=True)# df[f].astype(np.int64) * 1e-9
    df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']
    df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']

for f in ['feature_1','feature_2','feature_3']:
    order_label = df_train.groupby([f])['outliers'].mean()
    train_data[f] = df_train[f].map(order_label)
    test_data[f] = df_test[f].map(order_label)

In [ ]:
# There are duplicate card_ids in historical_data with 29,112,361 transactions, unique values are 325,540
# There are missing values in category_3', 'merchant_id' and 'category_2'

# Replace missing values with mode values for category_3', 'merchant_id' and 'category_2'
# When inplace = True, the data is modified in place,
# which means it will return nothing and the dataframe is now updated.

In [ ]:
# Need to impute values which is the challenge for prediction

# Stable documentation : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html#pandas.merge
# Merge train_data[['card_id','target']] with historical_data to give new historical_data
# merging target value of card_id for each transaction in historical_transactions Data; using union of keys from both frames
# historical_data = pd.merge(historical_data, train_data[['card_id','target']], how = 'outer', on = 'card_id')

# # merging target value of card_id for each transaction in new_merchants_transactions Data
# newmerchant_data = pd.merge(newmerchant_data, train_data[['card_id','target']], how = 'outer', on = 'card_id')

# historical_data['target'].isnull().sum() # / historical_data.shape[0] = 38.07%
# historical_data['target'].describe()

# for row in range(historical_data.shape[0]):
#   if (historical_data['card_id'][row] == test_data['card_id'][10]):
#     print(row)

# for card_id[0], 'C_ID_0ab67a22ab' : there are 1304310 - 1304243 = 67 historical transactions
# for card_id[10], 'C_ID_4859ac9ed5' : there are 23622204 - 23622180 = 24 historical transactions
# historical_data.loc[[1304243]]

# reset_index() to keep the 'card_id' column that appears with X.columns
# X = partial_historical_data_target.groupby('card_id').mean().reset_index()

# groupby_card_id.isnull().sum()

# Merge test_data[['card_id']] with partial_historical_data without target to give new test_data_intersect
# test_data_intersect = pd.merge(historical_data, test_data[['card_id']], how = 'inner', on = 'card_id')
# test_data_intersect

# Drop the non-numeric features
# partial_historical_data = historical_data_target.drop(columns=['target', 'authorized_flag', 'category_1', 'category_3', 'merchant_id', 'purchase_date'])

# historical_data['target'] = historical_data['target'].replace(0, historical_data['target'].mode())

<h2>Baseline Model :</h2>

In [ ]:
from sklearn.linear_model import LinearRegression
# Cost : Root Mean Square Error, RMSE
from sklearn.metrics import mean_squared_error
# Better Evaluation using cross-validation
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation:", scores.std())

# Linear Regression model
linear_reg = LinearRegression()
linear_reg.fit(X_train, data_labels)

# first_batchdata = X_train.iloc[:5]
# first_batchlabels = data_labels.iloc[:5]
# print("Predictions: ", linear_reg.predict(first_batchdata))
# print("Labels: ", list(first_batchlabels))

data_predictions = linear_reg.predict(X_train)
linear_mse = mean_squared_error(data_labels, data_predictions)
linear_rmse = np.sqrt(linear_mse)

linear_rmse # underfitting data; high bias

In [ ]:
%%time
# More Powerful Algorithm : DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_train[train_features], X_train['target'])

tree_reg_predictions = tree_reg.predict(X_train[train_features])
tree_mse = mean_squared_error(X_train['target'], tree_reg_predictions)
tree_rmse = np.sqrt(tree_mse)

print("DecisionTreeRegressor RMSE :", tree_rmse)

# scores = cross_val_score(tree_reg, X_train, data_labels,
#                          scoring="neg_mean_squared_error", cv=10)

# tree_rmse_scores = np.sqrt(-scores) # opposite of MSE, need to have negative sign

# # Different results when it is executed
# display_scores(tree_rmse_scores)

In [ ]:
X_train[train_features].shape

In [ ]:
card_ids = X_test["card_id"].copy()
tree_reg_predictions = tree_reg.predict(X_test[train_features])

create_file_for_submission("dtreereg_85_feats.csv", card_ids, tree_reg_predictions)

In [ ]:
%%time
# Ensemble Learning : RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

# The default value of n_estimators will change from 10 in version 0.20 to 100 in version 0.22
forest_reg = RandomForestRegressor(n_estimators=10)
forest_reg.fit(X_train[train_features], y)

data_predictions = forest_reg.predict(X_train[train_features])
forest_mse = mean_squared_error(y, data_predictions)
forest_rmse = np.sqrt(forest_mse)

print("RandomForestRegressor RMSE :", forest_rmse)

# forest_scores = cross_val_score(forest_reg, X_train, data_labels,
#                                 scoring="neg_mean_squared_error", cv=10)

# forest_rmse_scores = np.sqrt(-forest_scores) # opposite of MSE, need to have negative sign

# display_scores(forest_rmse_scores)

# Scores: [3.90089189 4.04644846 3.99770616 3.98337336 4.00308421 3.89694652
#  3.9277469  4.12725579 4.00782351 4.04242215]
# Mean: 3.993369895502748
# Standard Deviation: 0.06762168624862287
# CPU times: user 14min 51s, sys: 1.45 s, total: 14min 52s
# Wall time: 14min 53s

In [ ]:
%%time
# Search the best combination of hyperparameter values
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [10, 30], 'max_features': [10, 15, 20]}, # 'max_features': [2, 4, 6, 8]
    {'bootstrap': [False], 'n_estimators': [5, 15], 'max_features': [10, 20]}, # 'max_features': [2, 3, 4]
]

# The default value of n_estimators will change from 10 in version 0.20 to 100 in version 0.22
forest_reg = RandomForestRegressor(n_estimators=10) # no more warning on default 'n_estimators' value

grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring="neg_mean_squared_error")

grid_search.fit(X_train[train_features], y)

# CPU times: user 46min 45s, sys: 4.79 s, total: 46min 50s
# Wall time: 46min 50s

# GridSearchCV(cv=5, estimator=RandomForestRegressor(n_estimators=10),
#              param_grid=[{'max_features': [10, 15, 20],
#                           'n_estimators': [3, 10, 30]},
#                          {'bootstrap': [False], 'max_features': [9, 11, 17],
#                           'n_estimators': [3, 10]}],
#              scoring='neg_mean_squared_error')

# grid_search.best_params_
# {'max_features': 15, 'n_estimators': 30}

# GridSearchCV(cv=5, estimator=RandomForestRegressor(n_estimators=10),
#              param_grid=[{'max_features': [10, 15, 20, 30, 37],
#                           'n_estimators': [3, 10, 30]},
#                          {'bootstrap': [False],
#                           'max_features': [9, 11, 17, 27, 31],
#                           'n_estimators': [3, 10]}],
#              scoring='neg_mean_squared_error')

# grid_search.best_params_
# {'max_features': 10, 'n_estimators': 30}

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
cv_result = grid_search.cv_results_
for mean_score, params in zip(cv_result["mean_test_score"], cv_result["params"]):
    print(np.sqrt(-mean_score), params) # negative mean_score
    
# RMSE : 3.745822393200406 {'max_features': 10, 'n_estimators': 30} with 85 features

In [ ]:
# Analyze the Best Models and their Errors
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

In [ ]:
len(feature_importances)

In [ ]:
# Using GridSearchCV to obtain final model with best estimator
# final_model = grid_search.best_estimator_
rf_best = grid_search.best_estimator_

In [ ]:
rf_predictions = rf_best.predict(X_test[train_features])
rf_predictions.shape

In [ ]:
X[train_features].shape

In [ ]:
create_file_for_submission("rf_85_feats.csv", card_ids, rf_predictions)

## XGBoost Model

> https://xgboost.readthedocs.io/en/latest/parameter.html#

In [66]:
X_train.shape, X_test.shape

In [67]:
# Check for missing values : X_train.isna().any()
check_missing_values(X_train), check_missing_values(X_test)

In [ ]:
X_test.columns

In [ ]:
# X_test['new_merchant_purchase_date_min'].describe()
# X_test['new_merchant_purchase_date_min'].isna().value_counts()
# X_test['card_id'].isna().value_counts()

In [68]:
# Check for intersection between train features and missing values
intersect_features = set(train_features).intersection(check_missing_values(X_test)) # df_train
print("Common features in train and missing value test features (should be 0): {}".format(len(intersect_features)))
intersect_features

In [69]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import xgboost as xgb
np.random.seed(2728)

# Prepare data for training and prediction
X = X_train.copy()
y = X['target']
card_ids = X_test["card_id"].copy()

kfolds = KFold(n_splits=17, shuffle=True, random_state=2018)

print("kfolds :", kfolds)
# Make importance dataframe
importances = pd.DataFrame()

oof_preds = np.zeros(X.shape[0])
sub_preds = np.zeros(X_test.shape[0])

X.shape, y.shape, X_test.shape

In [70]:
# Difference in card_ids
col_difference = set(X.columns).symmetric_difference(X_test.columns)
print("Difference in train and test features: {}".format(len(col_difference)))
col_difference

In [71]:
np.isinf(X[train_features]).any().value_counts()

# print("printing column name where infinity is present")
# col_name = X[train_features].columns.to_series()[np.isinf(X[train_features]).any()]
# print(col_name)

In [72]:
np.isinf(X_test[train_features]).any().value_counts()

In [ ]:
print(df_train.shape, df_test.shape)
del df_train, df_test
gc.collect()

In [ ]:
%%time

## timeit will cause problem with importance
for n_fold, (trn_idx, val_idx) in enumerate(kfolds.split(X[train_features], y)): # X
    X_train_kf, y_train_kf = X[train_features].iloc[trn_idx], y.iloc[trn_idx]
    X_valid_kf, y_valid_kf = X[train_features].iloc[val_idx], y.iloc[val_idx]
    
    print("XGBR fold :", n_fold)
    print("=" * 80)
    
    # XGBoost Regressor estimator
    xgb_model = xgb.XGBRegressor(
        max_depth = 15,
        learning_rate = 0.01,
        n_estimators = 1000,
        subsample = .9,
        colsample_bylevel = .9,
        colsample_bytree = .9,
        min_child_weight= .9,
        gamma = 0,
        random_state = 100,
        booster = 'gbtree',
        objective = 'reg:squarederror' # 'reg:linear' deprecated
    )
    
    # Training
    xgb_model.fit(
        X_train_kf, y_train_kf,
        eval_set=[(X_train_kf, y_train_kf), (X_valid_kf, y_valid_kf)],
        verbose=True, eval_metric='rmse',
        early_stopping_rounds=100
    )
    
    # Feature importance
    imp_df = pd.DataFrame()
    imp_df['feature'] = train_features
    imp_df['gain'] = xgb_model.feature_importances_
    imp_df['fold'] = n_fold + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)
    
    oof_preds[val_idx] = xgb_model.predict(X_valid_kf)
    test_preds = xgb_model.predict(X_test[train_features])
    sub_preds += test_preds / kfolds.n_splits
    
    print("Next fold :", n_fold+1)
    print()
    
print("Final RMSE : ", np.sqrt(mean_squared_error(y, oof_preds)))

#### Validations with RMSE
>[324]	validation_0-rmse:1.78531	validation_1-rmse:3.75632</br>
> RMSE : 3.7005642695081877

>[334]	validation_0-rmse:1.75306	validation_1-rmse:3.75425</br>
> RMSE : 3.702697181424846

> [352]	validation_0-rmse:1.71289	validation_1-rmse:3.75283</br>
> RMSE : 3.7000391746720824</br>

> [357]	validation_0-rmse:1.64156	validation_1-rmse:3.74462</br>
> RMSE : 3.6941329565387844</br>

> [378]	validation_0-rmse:1.54589	validation_1-rmse:3.73675</br>
> Final RMSE :  3.6836558989291652

> [391]	validation_0-rmse:1.51994	validation_1-rmse:3.73630</br>
> Final RMSE :  3.6854681990466274</br>

> [385]	validation_0-rmse:1.61745	validation_1-rmse:3.73726</br>
> Final RMSE :  3.687870975295424</br>
> Wall time: 1h 19min 11s

> [645]	validation_0-rmse:1.31063	validation_1-rmse:3.90872</br>
> Kfold : 17
> Final RMSE :  3.679190047236234
> Wall time: 5h 28min 14s

#### Save trained ML models

In [ ]:
# Pickling files
print("Pickling one of the XGBRegressor model...")
filename = './XGBRegressor_model_pickle'
save_model_to_picklefile(filename, xgb_model)

In [ ]:
#### If XGBRegressor model exists, load it¶
retrieve_xgbr_file = './XGBRegressor_model_pickle'

if os.path.exists(retrieve_xgbr_file):
    # xgb_Classifier.save_model and load_model give an "le" error when trying to obtain score
    # Unpickling saved binary file if exist so that training do not need to done
    loaded_XGBRegressor = load_model_from_picklefile(retrieve_xgbr_file)
    print("Unpickling existing XGBRegressor model...")
    print("Loaded model :\n", loaded_XGBRegressor)
    print("with type\n", type(loaded_XGBRegressor))
    

xgbr_predictions = loaded_XGBRegressor.predict(X_test)

#### Feature Importances

In [ ]:
importances['gain_log'] = importances['gain']
mean_gain = importances[['gain', 'feature']].groupby('feature').mean()
importances['mean_gain'] = importances['feature'].map(mean_gain['gain'])

plt.figure(figsize=(16, 24))
sns.barplot(x='gain_log', y='feature', data=importances.sort_values('mean_gain', ascending=False))

plt.title('XGBRegressor Features (avg over folds)')
plt.tight_layout()
plt.savefig('xgbr_importances.png')

In [ ]:
# Length of submission
len(sub_preds), n_fold

In [ ]:
xgbr_predictions = sub_preds
xgbr_predictions.shape

In [ ]:
X[train_features].shape

In [ ]:
create_file_for_submission("xgbr_136_feats_KFold17.csv", card_ids, xgbr_predictions)

# Final model predictions for submission

In [ ]:
# len(feature_importance_df)
# feature_importance_df[["feature", "importance"]].sort_values(by="importance", ascending=False)[570:580]

# remove featuress to get better CV RMSE scores
# print("No. of train_features :", len(train_features))
# features_to_remove = ['new_merchant_card_id_size', 'new_merchant_merchants_active_months_lag12_mean_min',
#                       'new_merchant_merchants_active_months_lag3_sum_mean', 'new_merchant_merchants_active_months_lag6_sum_mean',
#                       'new_merchant_merchants_active_months_lag6_mean_sum', 'new_merchant_merchant_id_nunique',
#                       'new_merchant_year_nunique', 'new_merchant_weekend_sum',
#                       'hist_dayofweek_nunique', 'new_merchant_merchants_category_2_mean_max',
#                       'new_merchant_dayofweek_nunique', 'new_merchant_merchants_category_2_mean_min']

# train_features = list(set(train_features) - set(features_to_remove))

# print("No. of train_features after removing some low important features :", len(train_features))

# len(set(features_to_remove)), len(set(train_features))

In [ ]:
# train_features

# remove featuress to get better CV RMSE scores
# print("No. of train_features :", len(train_features))
# features_to_remove = ['f1_1', 'f1_2', 'f1_3', 'f1_4', 'f1_5',
#                       'f2_1', 'f2_2', 'f2_3', 'f3_0', 'f3_1']

# train_features = list(set(train_features) - set(features_to_remove))

# print("No. of train_features after removing some low important features :", len(train_features))

# len(set(features_to_remove)), len(set(train_features))

In [73]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
# import warnings
# warnings.filterwarnings('ignore')
np.random.seed(4590)
    
lgbm_params = {'num_leaves': 111,
               'min_data_in_leaf': 149,
               'objective':'regression',
               'max_depth': 9,
               'learning_rate': 0.005,
               "boosting": "gbdt",
               "feature_fraction": 0.7522,
               "bagging_freq": 1,
               "bagging_fraction": 0.7083,
               "bagging_seed": 11,
               "metric": 'rmse',
               "lambda_l1": 0.2634,
               "random_state": 133,
               "verbosity": -1}

In [ ]:
# lgbmreg_param = {'num_leaves': 111,
#                  'min_child_samples': 149,
#                  'max_depth': 9,
#                  'learning_rate': 0.005,
#                  "boosting_type": "gbdt",
#                  "reg_lambda": 0.2634,
#                  "random_state": 133,
#                  "verbosity": -1}

# lgb.LGBMRegressor.fit(X[train_features], y,
#                       num_round,
#                       eval_sets = [trn_data, val_data],
#                       verbose=100,
#                       early_stopping_rounds = 200)

In [ ]:
%%time
import time
categorical_feats = ['feature_1', 'feature_2', 'feature_3'] # 'feature_1', 
num_round = 10000

folds = KFold(n_splits=17, shuffle=True, random_state=15)
oof = np.zeros(len(X[train_features]))
lgbm_predictions = np.zeros(len(X_test[train_features]))
start = time.time()
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X[train_features].values, y.values)):
    print("LGBM fold n°{}".format(fold_))
    print("-" * 80)
    
    trn_data = lgb.Dataset(X.iloc[trn_idx][train_features],
                           label=y.iloc[trn_idx],
                           categorical_feature=categorical_feats)
    val_data = lgb.Dataset(X.iloc[val_idx][train_features],
                           label=y.iloc[val_idx],
                           categorical_feature=categorical_feats)

    # num_round = 10000
    lgbm_reg = lgb.train(lgbm_params,
                         trn_data,
                         num_round,
                         valid_sets = [trn_data, val_data],
                         verbose_eval=100,
                         early_stopping_rounds = 200)
    
    oof[val_idx] = lgbm_reg.predict(X.iloc[val_idx][train_features], num_iteration=lgbm_reg.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_features
    fold_importance_df["importance"] = lgbm_reg.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    lgbm_predictions += lgbm_reg.predict(X_test[train_features], num_iteration=lgbm_reg.best_iteration) / folds.n_splits
    print()

print("CV RMSE score: {:<8.5f}".format(mean_squared_error(oof, y)**0.5))

In [ ]:
cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')

In [80]:
lgbm_predictions.shape

In [81]:
X[train_features].shape

In [82]:
create_file_for_submission("lgbm_142_feats_merid_pdmax_median_KFold17.csv", card_ids, lgbm_predictions)

In [ ]:
from IPython.display import FileLink

FileLink(r'lgbm_85_feats_catid_median_KFold17.csv')

###### Download for submission :
> RandomForestRegressor with GridSearchCV using 3 features</br>
> Rank : 3719 out of 4110 [Private Score : 3.81301]</br>
> Rank : 3749 out of 4110 [Public Score : 3.93004]

> RandomForestRegressor(max_features=2, n_estimators=30) with some historical features</br>
> Rank : 3416 out of 4110 [Private Score : 3.79375]</br>
> Rank : 3238 out of 4110 [Public Score : 3.87633]

> XGBRegressor with 23 historical features</br>
> Rank : 3115 out of 4110 [Private Score : 3.73174] (75.7%)</br>
> Rank : 3104 out of 4110 [Public Score : 3.82532] (75.5%)

> XGBRegressor with 34 historical and new merchant features with mean_squared_error of 3.724</br>
> validation_0-rmse:1.94363	validation_1-rmse:3.77652</br>
> Rank : 3027 out of 4110 [Private Score : 3.70444] (73.65%)</br>
> Rank : 3016 out of 4110 [Public Score : 3.79859] (73.38%)

> XGBRegressor with 45 historical and new merchant features with RMSE of 3.7006.</br>
> validation_0-rmse:1.78531	validation_1-rmse:3.75632</br>
> Rank : 2757 out of 4110 [Private Score : 3.65051] (67.08%)</br>
> Rank : 2824 out of 4110 [Public Score : 3.74154] (68.71%)

> XGBRegressor with 55 historical (10 OHE for features1-3) and new merchant features with RMSE of 3.7027.</br>
> validation_0-rmse:1.75306	validation_1-rmse:3.75425</br>
> Rank : 2766 out of 4110 [Private Score : 3.65203] (67.30%)</br>
> Rank : 2816 out of 4110 [Public Score : 3.74019] (68.52%)

> XGBRegressor with 59 historical and new merchant features with RMSE of 3.6837.</br>
> validation_0-rmse:1.54589	validation_1-rmse:3.73675</br>
> Rank : 2755 out of 4110 [Private Score : 3.65021] (67.03%)</br>
> Rank : 2818 out of 4110 [Public Score : 3.74044] (68.56%)

> XGBRegressor with 69 historical and new merchant features with RMSE of 3.6837.</br>
> validation_0-rmse:1.60083	validation_1-rmse:3.73686</br>
> Final RMSE :  3.686220904191716</br>
> Rank : 2756 out of 4110 [Private Score : 3.65033] (67.06%)</br>
> Rank : 2781 out of 4110 [Public Score : 3.73777] (67.66%)

> XGBRegressor with 79 historical (10 OHE for features1-3) and new merchant features with RMSE of 3.7027.</br>
> validation_0-rmse:1.51994	validation_1-rmse:3.73630</br>
> Final RMSE :  3.6854681990466274</br>
> Rank : 2747 out of 4110 [Private Score : 3.64916] (66.84%)</br>
> Rank : 2756 out of 4110 [Public Score : 3.73545] (67.06%)</br>

> LGBM with 79 historical (10 OHE for features1-3) and new merchant features with RMSE of 3.6545.</br>
> [1041] training's rmse: 3.49544	valid_1's rmse: 3.60891
> CV score: 3.65450 </br>
> Rank : 2115 out of 4110 [Private Score : 3.62122] (51.46%)</br>
> Rank : 2370 out of 4110 [Public Score : 3.70550] (57.66%)</br>

> LGBM with 83 (10 OHE for features1-3 and new merchant purchase date min max mode) features at RMSE of 3.65311.</br>
> [1120] training's rmse: 3.4844	valid_1's rmse: 3.6065
> CV score: 3.65311 </br>
> Rank : 1947 out of 4110 [Private Score : 3.62025] (47.37%)</br>
> Rank : 2296 out of 4110 [Public Score : 3.70437] (55.86%)</br>

> LGBM with 87 using median for nan (new merchant purchase date min max mode with merchant data) features.</br>
> [1102] training's rmse: 3.47996	valid_1's rmse: 3.60637
> CV RMSE score: 3.65275</br>
> Rank : 1498 out of 4110 [Private Score : 3.61882] (36.45%)</br>
> Rank : 2227 out of 4110 [Public Score : 3.70239] (54.18%)</br>

> LGBM_KFold7 with 95_OHE using median for nan (new merchant purchase date min max mode with merchant data) features.</br>
> Rank : 1410 out of 4110 [Private Score : 3.61813] (34.31%)</br>
> Rank : 2185 out of 4110 [Public Score :  3.70141] (53.16%)</br>
> CV RMSE score: 3.65106

> LGBM_KFold17 with 128 features using median for nan (new merchant purchase date min max mode with merchant data).</br>
> Rank : 1805 out of 4110 [Private Score : 3.61959] (43.92%)</br>
> Rank : 2184 out of 4110 [Public Score :  3.70135] (53.14%)</br>
> CV RMSE score: 3.64977

> LGBM_KFold17 with 136 features using median for nan (new merchant purchase date min max mode with merchant data).</br>
> Rank : 1655 out of 4110 [Private Score : 3.61937] (40.27%)</br>
> Rank : 2160 out of 4110 [Public Score :  3.70097] (52.55%)</br>
> CV RMSE score: 3.64882
> Wall time: 33min 52s

> LGBM_KFold17_merchantid with 137 features using median for nan (new merchant purchase date min max mode with merchant data, purchase date max ratio) at CV RMSE of 3.64613.</br>
> Rank : 963 out of 4110 [Private Score : 3.61572] (23.43%)</br>
> Rank : 1722 out of 4110 [Public Score :  3.69594] (41.90%)</br>
> CV RMSE score: 3.64613

> LGBM_KFold17_merchantid with 137 features using median for nan (new merchant purchase date min max mode with merchant data, purchase date max ratio, pd_max) at CV RMSE of 3.64625.</br>
> Rank : 922 out of 4110 [Private Score : 3.61546] (22.43%)</br>
> Rank : 1672 out of 4110 [Public Score :  3.69539] (40.68%)</br>
> CV RMSE score: 3.64625
> Early stopping, best iteration is:
> [1152]	training's rmse: 3.47992	valid_1's rmse: 3.43485
> Wall time: 49min 54s

> LGBM_KFold17_merchantid with 137 features using median for nan (new merchant purchase date min max mode with merchant data, 'new_hist_purchase_date_max_ratio', pd_max, 'new_hist_purchase_amount_max_ratio') at CV RMSE of 3.64680.</br>
> Rank : 352 out of 4110 [Private Score : 3.61319] (8.56%)</br>
> Rank : 1560 out of 4110 [Public Score :  3.69400] (37.96%)</br>
> CV RMSE score: 3.64680
> Early stopping, best iteration is:
> [1263]	training's rmse: 3.46726	valid_1's rmse: 3.43649
> Wall time: 37min 10s

[1400]	training's rmse: 3.45464	valid_1's rmse: 3.43656

## Not improving
> Remove the 'new_merchant_purchase_date_max' and 'hist_purchase_date_max' after creating the ratio
> Early stopping, best iteration is:
> [1007]	training's rmse: 3.49581	valid_1's rmse: 3.43503
> CV RMSE score: 3.64655 
> Wall time: 52min 27s

In [ ]:
from sklearn.ensemble import VotingRegressor

# import xgboost as xgb
# import lightgbm as lgb
# from sklearn.tree import DecisionTreeRegressor

# # Train classifiers
# xgb_model = xgb.XGBRegressor(
#     max_depth = 15,
#     learning_rate = 0.01,
#     n_estimators = 1000,
#     subsample = .9,
#     colsample_bylevel = .9,
#     colsample_bytree = .9,
#     min_child_weight= .9,
#     gamma = 0,
#     random_state = 100,
#     booster = 'gbtree',
#     objective = 'reg:squarederror' # 'reg:linear' deprecated
# )

# param = {'num_leaves': 111,
#          'min_data_in_leaf': 149, 
#          'objective':'regression',
#          'max_depth': 9,
#          'learning_rate': 0.005,
#          "boosting": "gbdt",
#          "feature_fraction": 0.7522,
#          "bagging_freq": 1,
#          "bagging_fraction": 0.7083 ,
#          "bagging_seed": 11,
#          "metric": 'rmse',
#          "lambda_l1": 0.2634,
#          "random_state": 133,
#          "verbosity": -1}

reg1 = xgb_model
reg2 = lgbm_reg
reg3 = tree_reg

ereg = VotingRegressor([('gb', reg1), ('lr', reg3)]) # ('rf', reg2)
ereg

# from sklearn.linear_model import Lasso
# lasso_reg = Lasso(alpha=0.1)
# lasso_reg.fit(X[train_features], y)

In [ ]:
%%time
ereg.fit(X[train_features], y)

In [ ]:
%%time
ensemble_predictions  = ereg.predict(X_test[train_features])
ensemble_predictions.shape

In [ ]:
create_file_for_submission("xgbr_treereg_95_feats_median_KFold7.csv", card_ids, ensemble_predictions)

In [83]:
!ls -tl

### Feature Extractions (with Sequential Feed-forward Neural Network)
* Train our model</br>
> For training a model, we use the fit function, which trains the model for a given number of epochs, which refers to the number of times we pass our dataset through our model to train it.  We use callbacks.TensorBoard to writes a log for TensorBoard, which allows you to visualize dynamic graphs of your training and test metrics, as well as activation histograms for the different layers in your model.  To save our model after every epoch we using callbacks.ModelCheckpoint for it.

In [ ]:
# excluded_features = ['first_active_month', 'card_id', 'target', 'hist_purchase_date_min', 'hist_purchase_date_max',
#                      'new_merchant_purchase_date_min', 'new_merchant_purchase_date_max',
#                      'feature_1', 'feature_2', 'feature_3', 'hist_dayofweek_nunique', 'hist_hour_nunique',
#                      'new_merchant_weekend_sum', 'new_merchant_weekend_mean', 'hist_month_nunique',
#                      'new_merchant_category_3_mean_mean', 'hist_merchant_id_nunique',
#                      'hist_purchase_amount_min', 'hist_weekofyear_nunique', 'hist_installments_max',
#                      'hist_purchase_amount_max', 'hist_category_2_mean_mean'
#                     ]
# train_features = [c for c in df_train.columns if c not in excluded_features]

# print("Features used for training : ", train_features)

In [ ]:
# 43 out of 49 features
X[train_features].shape, y.shape

In [ ]:
df_train.columns

In [ ]:
# tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("weights_{epoch:02d}_{val_loss:.2f}.h5", monitor='val_loss',
                                              verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [ ]:
def build_model(input_df):
    print("No. of features :", input_df.shape[1])
    model = models.Sequential()
    model.add(layers.Dense(1024, activation='relu', input_shape=(input_df.shape[1],)))
    model.add(layers.Dropout(0.2)),
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.3)),
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(0.5)),
    model.add(layers.Dense(1))

    lr_schedule = ExponentialDecay(initial_learning_rate=1e-3,
                                   decay_steps=10000, decay_rate=0.9)
    
    model.compile(optimizer=Adam(learning_rate=1e-3), # Adadelta(learning_rate=0.001, rho=0.95, epsilon=1e-07, name="Adadelta")
                  loss='mean_squared_error', # mse
                  metrics=['RootMeanSquaredError'])
    return model

In [ ]:
# import keras
# from keras import callbacks
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, BatchNormalization
# from tensorflow.keras import models, layers
# # from keras.optimizers import RMSprop, SGD
# from keras.optimizers import Adam, Adadelta
# from keras.optimizers.schedules import ExponentialDecay
# from keras.metrics import RootMeanSquaredError

def build_model(input_df):
    print("No. of features :", input_df.shape[1])
    model = models.Sequential()

    model.add(Dense(2 ** 10, input_dim = input_df.shape[1],
                    kernel_initializer='glorot_uniform', activation='relu')) # init='random_uniform',
    model.add(Dropout(0.25))    
    model.add(BatchNormalization())
    model.add(Dense(2 ** 9, activation='relu')) # kernel_initializer='random_uniform'
    model.add(BatchNormalization())
    model.add(Dropout(0.25)) 
    model.add(Dense(2 ** 5, activation='relu')) # kernel_initializer='random_uniform'
    model.add(BatchNormalization())
    model.add(Dropout(0.25))      
    model.add(Dense(1))

    #     model.compile(loss='mean_squared_error', optimizer='adam')
    model.compile(optimizer=Adam(learning_rate=1e-3), # Adadelta(learning_rate=0.001, rho=0.95, epsilon=1e-07, name="Adadelta")
                  loss='mse',
                  metrics=['RootMeanSquaredError'])
    return model

In [ ]:
dl_model = build_model(X[train_features])

dl_model.summary()

In [ ]:
model_log = dl_model.fit(X[train_features], y, epochs=10, batch_size=64, # 16
                         validation_split = 0.2,
                         callbacks=[model_checkpoints]) # , verbose=0)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2,figsize=(15,5))

ax1.plot(model_log.history['root_mean_squared_error'])
ax1.plot(model_log.history['val_root_mean_squared_error'])
ax1.set_title('Root Mean Squared Error (RMSE)')
ax1.set(xlabel='Epoch', ylabel='RMSE')
ax1.legend(['train', 'validation'], loc='upper right')
ax1.xaxis.set_major_locator(MaxNLocator(integer=True))

ax2.plot(model_log.history['loss'])
ax2.plot(model_log.history['val_loss'])
ax2.set_title('Loss (Lower Better)')
ax2.set(xlabel='Epoch', ylabel='Loss')
ax2.legend(['train', 'validation'], loc='upper right')
ax2.xaxis.set_major_locator(MaxNLocator(integer=True))

In [ ]:
first_batchdata = X[train_features].iloc[:5]
first_batchlabels = y.iloc[:5]

print("XBGR Predictions: ", xgb_model.predict(first_batchdata))
print("LGBM Predictions: ", lbg_reg.predict(first_batchdata))
print("DL Predictions: ", dl_model.predict(first_batchdata))

In [ ]:
print("Labels: ", list(first_batchlabels))

test_mse_score, test_rmse_score = dl_model.evaluate(first_batchdata, first_batchlabels)

print(f'test_mse_score : {test_mse_score}')
print(f'test_rmse_score : {test_rmse_score}')

# The attribute model.metrics_names will give you the display labels for the evalution outputs.
print("\nModel metrics :", dl_model.metrics_names)

In [ ]:
df_test[train_features].shape

In [ ]:
dl_predictions = dl_model.predict(df_test[train_features])
dl_predictions.shape, df_test.shape, df_test[train_features].shape

In [ ]:
create_file_for_submission("DL5_141feat_adam_rmse_128NN.csv", card_ids, dl_predictions[:, 0])

In [ ]:
xgbr_predictions_df = pd.DataFrame(xgbr_predictions, columns = ['XGBR_Pred'])
xgbr_predictions_df

In [ ]:
lgbm_predictions_df = pd.DataFrame(lgbm_predictions, columns = ['LGBM_Pred'])
lgbm_predictions_df

In [ ]:
dtreg_predictions_df = pd.DataFrame(tree_reg_predictions, columns = ['DTR_Pred'])
dtreg_predictions_df

In [ ]:
concat_pred_df = pd.concat([xgbr_predictions_df, lgbm_predictions_df, dtreg_predictions_df], axis=1, sort=False)
concat_pred_df

In [ ]:
rfb_predictions_df = pd.DataFrame(rf_predictions, columns = ['RFB_Pred'])
rfb_predictions_df

In [ ]:
concat_pred_df = pd.concat([concat_pred_df, rfb_predictions_df], axis=1, sort=False)
concat_pred_df

In [ ]:
concat_pred_df

In [ ]:
concat_pred_df.drop(columns=['Avg_Pred', 'DTR_Pred'], axis=1, inplace=True)
concat_pred_df

In [ ]:
dl_predictions_df = pd.DataFrame(dl_predictions[:, 0], columns = ['DL_Pred'])
dl_predictions_df

In [ ]:
type(concat_pred_df), type(dl_predictions_df)

In [ ]:
concat_pred_df = pd.concat([concat_pred_df, dl_predictions_df], axis=1, sort=False)
concat_pred_df

In [ ]:
concat_pred_df['Avg_Pred'] = concat_pred_df.mean(axis=1)
concat_pred_df

In [ ]:
concat_pred_df['Avg_Pred'].shape

In [ ]:
X[train_features].shape

In [ ]:
# Average predicted values from XGBR, LGB and DL
create_file_for_submission("xgbr_lgbm_rfb_85feat.csv", card_ids, concat_pred_df['Avg_Pred'])

#### Ensemble Learning with weights (Attempt novelty)

In [ ]:
# Load from saved files
# xgbr_preds = pd.read_csv('../input/testpredictions/xgbr_79_feats.csv')
# lgbm_preds = pd.read_csv('../input/testpredictions/lgb_79_feats.csv')
# dl_preds = pd.read_csv('../input/testpredictions/xgbr_lgbm_dl_79feat.csv')

# concat_pred_df = pd.concat([xgbr_preds['target'], lgbm_preds['target'], dl_preds['target']],
#                             axis=1, keys=['XGBR_Pred', 'LGBM_Pred', 'DL_Pred'])

# concat_pred_df['Avg_Pred'] = concat_pred_df.mean(axis=1)

# xgbr_preds.loc[:5, 'target'], lgbm_preds.loc[:5, 'target'], dl_preds.loc[:5, 'target']
# concat_pred_df

In [ ]:
xgbr_predictions.shape, lgbm_predictions.shape, tree_reg_predictions.shape, rf_predictions.shape

In [ ]:
concat_pred_df.head(2)

In [ ]:
ensemble_reg_predictions_df = pd.DataFrame(ensemble_reg_pred, columns = ['ESMR_Pred'])
ensemble_reg_predictions_df

In [ ]:
no_of_samples = 20
xgbr_pred_samples = concat_pred_df.loc[:no_of_samples, 'XGBR_Pred']
lgbm_pred_samples = concat_pred_df.loc[:no_of_samples, 'LGBM_Pred']
rfb_pred_samples = concat_pred_df.loc[:no_of_samples, 'RFB_Pred']
# dtreg_pred_samples = concat_pred_df.loc[:no_of_samples, 'DTR_Pred']

model_avg_pred_samples = concat_pred_df.loc[:no_of_samples, 'Avg_Pred']
esm_reg_pred_samples = ensemble_reg_predictions_df.loc[:no_of_samples, 'ESMR_Pred']

# https://matplotlib.org/stable/tutorials/colors/colors.html
# https://matplotlib.org/stable/api/markers_api.html

plt.figure(figsize=(20,12))
plt.plot(xgbr_pred_samples, 'gd', label='XGBR')
plt.plot(lgbm_pred_samples, 'b^', label='LGBM')
plt.plot(dtreg_pred_samples, 'ys', label='RFB') # DL5
plt.plot(model_avg_pred_samples, 'r*', ms=10, label='Average')

plt.plot(esm_reg_pred_samples, 'm.', ms=10, label='Average')

plt.tick_params(axis='x', which='both', bottom=False, top=False,
                labelbottom=False)

plt.ylabel('predicted')
plt.xlabel('Test samples')
plt.legend(loc="best")
plt.title('Model predictions and their average')

plt.show()

In [ ]:
no_of_samples = 20
xgbr_pred_samples = xgbr_preds.loc[:no_of_samples, 'target']
lgbm_pred_samples = lgbm_preds.loc[:no_of_samples, 'target']
dl_pred_samples = dl_preds.loc[:no_of_samples, 'target']
model_avg_pred_samples = concat_pred_df.loc[:no_of_samples, 'Avg_Pred']

plt.figure(figsize=(20,12))
plt.plot(xgbr_pred_samples, 'gd', label='XGBR')
plt.plot(lgbm_pred_samples, 'b^', label='LGBM')
plt.plot(dl_pred_samples, 'ys', label='DL5')
plt.plot(model_avg_pred_samples, 'r*', ms=10, label='Average')

plt.tick_params(axis='x', which='both', bottom=False, top=False,
                labelbottom=False)

plt.ylabel('predicted')
plt.xlabel('Test samples')
plt.legend(loc="best")
plt.title('Model predictions and their average')

plt.show()

In [ ]:
%%time
xgbr_train_pred = xgb_model.predict(X[train_features])
lgbm_train_pred = lgbm_reg.predict(X[train_features])
rfb_train_pred = rf_best.predict(X[train_features])

# dtreg_train_pred = tree_reg.predict(X[train_features])

# dl_train_pred = dl_model.predict(X[train_features])

xgbr_train_pred_df = pd.DataFrame(xgbr_train_pred, columns = ['XGBR_Tr_Pred'])
lgbm_train_pred_df = pd.DataFrame(lgbm_train_pred, columns = ['LGBM_Tr_Pred'])
rfb_train_pred_df = pd.DataFrame(rfb_train_pred, columns = ['RFB_Tr_Pred'])

# dtreg_train_pred_df = pd.DataFrame(dtreg_train_pred, columns = ['DTR_Tr_Pred'])

# xgbr_train_pred_df, lgbm_train_pred_df, dtreg_train_pred_df and dl_train_pred_df
concat_train_pred = pd.concat([xgbr_train_pred_df, lgbm_train_pred_df, rfb_train_pred_df], axis=1, sort=False)
concat_train_pred

In [ ]:
%%time
# Linear Regression model to have ensemble regression
from sklearn.tree import DecisionTreeRegressor
# Cost : Root Mean Square Error, RMSE
from sklearn.metrics import mean_squared_error

tree_reg = DecisionTreeRegressor()
tree_reg.fit(concat_train_pred, y)

train_pred = tree_reg.predict(concat_train_pred)
tree_mse = mean_squared_error(y, train_pred)
tree_rmse = np.sqrt(tree_mse)

print("DecisionTreeRegressor RMSE :", tree_rmse)

# scores = cross_val_score(tree_reg, concat_train_pred, y,
#                          scoring="neg_mean_squared_error", cv=10)

# tree_rmse_scores = np.sqrt(-scores) # opposite of MSE, need to have negative sign

# # Different results when it is executed
# display_scores(tree_rmse_scores)

In [ ]:
test_pred = concat_pred_df.drop('Avg_Pred', axis=1)
test_pred

In [ ]:
ensemble_reg_pred = tree_reg.predict(test_pred)
ensemble_reg_pred.shape

In [ ]:
X[train_features].shape

In [ ]:
# Using a trained linear regressor with predicted values from XGBR, LGB and DL
create_file_for_submission("xgbr_lgbm_rfb_85feat_reg.csv", card_ids, ensemble_reg_pred)

In [ ]:
# train_targets = train_data["target"].copy()
# df_train = train_data.drop(columns=['card_id', 'first_active_month', 'target'])

# df_train.shape, train_targets.shape

# --
# X_test = test_data.drop(columns=['card_id', 'first_active_month'])


# --
# card_ids = test_data["card_id"].copy()

# card_ids.shape
# --
# kaggle = pd.DataFrame({'card_id': card_ids, 'target': xtest_predictions[:, 0]})
# kaggle.to_csv('./elo_dl_pred.csv', index=False)

In [ ]:
# Download csv file
# <a href="./elo_pred.csv"> Download File </a>

<h2>Exploring the train and test data files :</h2>

In [ ]:
# In order to read .xlsx file
# !pip install openpyxl
# Read excel-formatted data dictionary file with pandas
# data_dictionary=pd.read_excel('../input/elo-merchant-category-recommendation/Data Dictionary.xlsx')
# data_dictionary

**Observations :**

* This DataDictionary file have the description of all the features in Description column which were included in train.csv.

* From second row we have columns which have the description of all the columns in our data and third row tell us about the card_id and third one is about the first_active_month which tell us about the month and year of purchase of products.

* feature_1, feature_2, feature_3 has categorical value which is in row fourth,fifth,and sixth.

* last row tells us about the prediction on the basis of these features which is known as target column. or we can say loyalty score which is calculated after the two months.

In [ ]:
print('The number of rows in train_data is:',train_data.shape[0])
print('The number of rows in test_data is:',test_data.shape[0])
plt.bar([0,1],[train_data.shape[0],test_data.shape[0]])
plt.xticks([0,1],['train_rows','test_rows'])

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.info()
print("********************************************************************")
test_data.info()

**Obsaervations :**

* The main data train has 6 values. 'first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3', 'target'.
* first_active_month : This is active_month for card_id. 
* feature_1,2,3 : it is key important but hidden meaning.
* target : Loyalty numerical score calculated 2 months after historical and evaluation period
* We can infer that both the data have same columns and overall same structure. So, We will explore both data simultaneously.


**Missing values in train and test data :
(Check for nan values in the whole train and test data)**

In [ ]:
train_data.isna().any()

**Observation :** In train Data there is no nan values for any features in train data

In [ ]:
test_data.isna().any()

In [ ]:
test_data[test_data['first_active_month'].isna()]

**Observation :**
In the Test Data, there is one row with 'first_active_month' as nan value. Since it is test data we have to impute the value.

**Feature comparison in train and test data features :**

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (15, 5));
train_data['feature_1'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='feature_1', rot=0);
train_data['feature_2'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='feature_2', rot=0);
train_data['feature_3'].value_counts().sort_index().plot(kind='bar', ax=ax[2], color='gold', title='feature_3', rot=0);
plt.suptitle('Counts of categories for train features');

fig, ax = plt.subplots(1, 3, figsize = (15, 5));
test_data['feature_1'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='feature_1', rot=0);
test_data['feature_2'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='feature_2', rot=0);
test_data['feature_3'].value_counts().sort_index().plot(kind='bar', ax=ax[2], color='gold', title='feature_3', rot=0);
plt.suptitle('Counts of categories for test features');

**Observations :**

* We can see from above plots that test and train data are distributed similarly.
* feature_1, feature_2, feature_3, all are categorical variables
* feature_1 has 5 unique values
* feature_2 has 3 unique values
* feature_3 is a binary column

In [ ]:
# No. of unique values; not all target values are unique (total number : 201917)
train_data['feature_1'].nunique(), train_data['feature_2'].nunique(), train_data['feature_3'].nunique(), train_data['target'].nunique()

**Anonymised Features Analysis : feature_1, feature_2, feature_3**

**checking distributions with target :**

In [ ]:
plt.figure(figsize=(20,5))
plt.subplot(131)
sns.kdeplot(x ='target',data = train_data,hue = 'feature_1',palette='rainbow')
plt.title('Pdf of target over different categories of Feature_1')
plt.subplot(132)
sns.kdeplot(x ='target',data = train_data,hue = 'feature_2',palette='Dark2_r')
plt.title('distribution of target over different categories of Feature_2')
plt.subplot(133)
sns.kdeplot(x ='target',data = train_data,hue = 'feature_3',palette='Dark2_r')
plt.title('distribution of target over different categories of Feature_3')
plt.show()

**Observations :** 

The above two plots show a key point : 

* while different categories of these features could have various counts, the distribution of target is almost the same. This could mean, that these features aren't really good at predicting target - we'll need other features and feature engineering. Also it is worth noticing that mean target values of each catogory of these features is near zero. This could mean that data was sampled from normal distribution.

**Note:** The same information can be gathered by using box-plot and violin-plot, I have tried all of them. Here, I use kdeplot as I found it more visually appealing. In further analysis I have used Box-plot more often.

**let's see Target column seperately :**

In [ ]:
train_data['target'].describe()

In [ ]:
#Plotting the estimated pdf of target variable with kernel density estimation (KDE)
sns.kdeplot(train_data['target'])
plt.title("PDF of Target")
plt.show()

**Observation:** The target value is almost normally distributed with bunch of outlier value near -30. This distribution indicates that the target value is normalized with pre-decided mean and standard deviation.

This outlier value of the target is a value which needs more look into the feature EDA to understand cause of it.

**Analyze the outliers :**



In [ ]:
loyality_score = train_data['target']
ax = loyality_score.plot.hist(bins=20, figsize=(6, 5))
_ = ax.set_title("target histogram")
plt.show()

fig, axs = plt.subplots(1,2, figsize=(12, 5))
_ = loyality_score[loyality_score > 10].plot.hist(ax=axs[0])
_ = axs[0].set_title("target histogram for values greater than 10")
_ = loyality_score[loyality_score < -10].plot.hist(ax=axs[1])
_ = axs[1].set_title("target histogram for values less than -10")
plt.show()


**Observations :**

* Values range from -33.2 to 17.9

* -33 seems like an outlier as can be seen in the 3rd plot

* other values less than -10 also seem like outliers due to very less in number

* All values above 10 are also looking like outliers

In [ ]:
target_sign = loyality_score.apply(lambda x: 0 if x <= 0 else 1)
target_sign.value_counts()

**Observation :** Negative and positive target values are almost in the same proportion

In [ ]:
outliers_in_target= train_data.loc[(train_data['target']< -10) | (train_data['target']>10)]
print(' The number of outliers in the data is:',outliers_in_target.shape[0])
non_outliers_in_target= train_data.loc[(train_data['target'] >=-10) & (train_data['target']<=10)]
print(' The number of non-outliers in the data is:',non_outliers_in_target.shape[0])

Outliers comparison with the feature of target :

In [ ]:
plt.figure(figsize=[16,9])
plt.suptitle('Outlier vs. non-outlier feature distributions', fontsize=20, y=1.1)

for num, col in enumerate(['feature_1', 'feature_2', 'feature_3', 'target']):
    if col is not 'target':
        plt.subplot(2, 3, num+1)
        non_outlier = non_outliers_in_target[col].value_counts() / non_outliers_in_target.shape[0]
        plt.bar(non_outlier.index, non_outlier, label=('non-outliers'), align='edge', width=-0.3, edgecolor=[0.2]*3,color=['teal'])
        outlier = outliers_in_target[col].value_counts() / outliers_in_target.shape[0]
        plt.bar(outlier.index, outlier, label=('outliers'), align='edge', width=0.3, edgecolor=[0.2]*3,color=['brown'])
        plt.title(col)
        plt.legend()

plt.tight_layout()
plt.show()

**Observations :**

* We can see There are only slight differences between outliers and non-outliers, but they don't seem to be that big and they certainly can't explain the difference between the target values, at least based on the features in the train dataset. It means the card_id's having outliers as loyality score having pretty much similar properties to the regular ones.

* Outliers could be one of the main purposes of this competition. May be those represent fraud or credit default etc. i.e. they are important. The target variable is normally distributed, and outliers seem to be purposely introduced in the loyalty formula. 

* As noted in multiple threads over kaggle, more than half of the RMSE is due to the outliers with loyalty scores of ~ -33. They strongly mention, If we try to replace these outliers with the median, retrain the model and submit, we will
find our leaderboard score WORSE than if we keep the outliers at their original values. Impute any values will significantly affect the RMSE score for test set. So, imputations have been excluded. This tells us that outliers are included in the test set. Furthermore, given the magnitude of the impact of outliers on the RMSE score, Our focus should be on predicting those outliers as accurately as possible.

* For mitigating the impact of outliers, We can make the outliers as a binary feature whether card's target value is outliers or not. So that while training our model can learn that given entry has target score as outlier or not and use this information while predicting loyality score.

**Analysis of feature First_active_month :**

Distribution of first_active_month across years :

In [ ]:
year_train = train_data['first_active_month'].value_counts().sort_index()
year_test = test_data['first_active_month'].value_counts().sort_index()
ax = year_train.plot(figsize=(10, 5))
ax = year_test.plot(figsize=(10, 5))
_ = ax.set_xticklabels(range(2010, 2020))
_ = ax.set_title("Distribution across years")
_ = ax.legend(['train', 'test'])

**Observation :** Years range from 2011 to 2018. But, Most of the data lies in the years ranging from 2016 to 2018 and trends of counts for train and test data are similar.

Distribution of first_active_month across months :

In [ ]:
train_data['first_active_month'].value_counts()

In [ ]:
train_data['first_active_month'].head()

In [ ]:
# Feature Extraction
train_data["month"] = train_data['first_active_month'].str.split("-").str[1]
train_data.head()

In [ ]:
temp = train_data['month'].value_counts().sort_index()
ax = temp.plot()
_ = ax.set_xticklabels(range(-1, 15, 2))
_ = ax.set_title("Distribution across months")

**Observations :** Last 6 months (July to December) has relatively more data than first 6 months (January to June).

**First_active_month Vs Target variable :**

In [ ]:
train_data['first_active_month'] = pd.to_datetime(train_data['first_active_month'],
                                                  format='%Y-%m')

In [ ]:
train_data['first_active_month'].head()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 8))

sns.boxplot(x=train_data["target"])

**Correlation between variables : Pearson Correlation (Bi-variate)**

In [ ]:
features_correlation = pd.DataFrame(train_data, columns=['feature_1', 'feature_2', 'feature_3', 'target'])

colormap = plt.cm.RdBu
plt.figure(figsize=(10,8))
plt.title('Pearson Correlation of anonymise Features with Target', y=1.05, size=15)
sns.heatmap(features_correlation.astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)

In [ ]:
sns.lineplot(x = train_data['first_active_month'], y= train_data['target'])
plt.title("Distribution of target over first_active_month")
plt.show()

**Observations :**

* The above plot reveals that the target variable (loyalty score) behaves like a damping frequency plot. And it is mentioned in the Buisness problem that the target score is calcuated with the recent year transactions.

* Older Card's: The cards which have first active month from 2012 to 2015.

* new card's: The cards which have first active month from 2015 to 2018.

* The Older card's have large number of transactions which affects the target towards the negative value. and the new card's have transactions which affects the target towards positive value.

So, I think the type of transactions by the newer card's is different from the older card's which helps in increase the loyalty Score.



**Correlation between variables : Variance Inflation Factor (Multicollinearity)**

Reference : https://www.statisticshowto.com/variance-inflation-factor

**Abstract :**

Multicollinearity occurs when two or more independent variables are highly correlated with one another in a regression model.  Multicollinearity can be a problem in a regression model because we would not be able to distinguish between the individual effects of the independent variables on the dependent variable.

A variance inflation factor(VIF) detects multicollinearity in regression analysis. Multicollinearity is when there’s correlation between predictors (i.e. independent variables) in a model; it’s presence can adversely affect your regression results. The VIF estimates how much the variance of a regression coefficient is inflated due to multicollinearity in the model.

- VIF determines the strength of the correlation between the independent variables. It is predicted by taking a variable and regressing it against every other variable.
- VIF score of an independent variable represents how well the variable is explained by other independent variables.
- VIF = 1 / (1 - R^2)
- R^2 means that the variable is highly correlated with the other variables
- The closer R^2 value is to 1, the higher the value of VIF and the higher the multicollinearity with the particular independent variable
- Although correlation matrix and scatter plots can also be used to find multicollinearity, their findings only show the bivariate relationship between the independent variables.  VIF is preferred as it can show the correlation of a variable with a group of other variables.
- Dropping variables should be an iterative process starting with the variable having the largest VIF value because its trend is highly captured by other variables. If you do this, you will notice that VIF values for other variables would have reduced too, although to a varying extent
- When you care more about how much each individual feature rather than a group of features affects the target variable, then removing multicollinearity may be a good option
- If multicollinearity is not present in the features you are interested in, then multicollinearity may not be a problem
- Knowledge about multicollinearity can be quite helpful when you’re building interpretable machine learning models

A rule of thumb for interpreting the variance inflation factor:

1 = not correlated.
Between 1 and 5 = moderately correlated.
Greater than 5 = highly correlated.

Exactly how large a VIF has to be before it causes issues is a subject of debate. What is known is that the more your VIF increases, the less reliable your regression results are going to be. In general, a VIF above 10 indicates high correlation and is cause for concern. Some authors suggest a more conservative level of 2.5 or above.

In [ ]:
#Finding Correlation between variables of train_data features
selected_columns = ['feature_1','feature_2','feature_3']
data_frame = train_data[selected_columns]

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(data_frame.iloc[:,:].values, i) for i in range(data_frame.shape[1])]
vif["features"] = data_frame.columns
vif

In [ ]:
#Finding Correlation between variables of test_data features
selected_columns = ['feature_1','feature_2','feature_3']
data_frame = test_data[selected_columns]

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(data_frame.iloc[:,:].values, i) for i in range(data_frame.shape[1])]
vif["features"] = data_frame.columns
vif

**Observations :** 

* The VIF values for all the three features are well under 10. So, there is no problem of multicollinearity in the train data and test data.
* Also VIF values are very near to 0, which interpret that features are not at all correlated.

<h2>Exploring the historical_transactions and new_merchant_transactions data files :</h2>

In [ ]:
# Read excel-formatted data dictionary file with pandas
#data_dictionary = pd.read_excel('../input/elo-merchant-category-recommendation/Data_Dictionary.xlsx', sheet_name='history')
#data_dictionary

In [ ]:
# Read excel-formatted data dictionary file with pandas
#data_dictionary = pd.read_excel('../input/elo-merchant-category-recommendation/Data_Dictionary.xlsx', sheet_name='new_merchant_period')
#data_dictionary

**Observation :** After going through "history" and "new_merchant_period" worksheets in Data Dictionary.xlsx, We can infer that both the data have same columns and overall same structure. So, We will Explore both data side by side.

In [ ]:
print(f'{historical_data.shape[0]} rows in historical transactions!\n')
historical_data.head()

In [ ]:
print(f'{newmerchant_data.shape[0]} rows in new merchants data!\n')
newmerchant_data.head()

In [ ]:
historical_data.describe()

**Observations :**

We can see that there are:

* 6 features type ID: card_id, merchant_category_id, subsector_id, merchant_id, city_id, state_id

* 2 features type integer/counter: month_lag, installments

* 1 feature type numerical: purchase_amount

* 1 feature type date: purchase_date

* 4 features type categorical: authorized_flag, category_3, category_1, category_2

In [ ]:
# By default, Non-Null Counts are shown only if the DataFrame is smaller than
# pandas.options.display.max_info_rows and pandas.options.display.max_info_columns.
historical_data.info(show_counts=True)

In [ ]:
# category_3, merchant_id and category_2 have null values
historical_data.isna().any()

In [ ]:
newmerchant_data.info(show_counts=True)

In [ ]:
# category_3, merchant_id and category_2 have null values
newmerchant_data.isna().any()

**Observations :** Both historical_transaction and new_merchant_transaction have Nan values in same columns which are : merchand_id, category_2, category_3.

**Analysis of Category Features : category_1,category_2 and category_3**

In [ ]:
print('Value counts for category features of Historical Transactions :\n')
print(historical_data['category_1'].value_counts())
print('*****************************')
print(historical_data['category_2'].value_counts())
print('*****************************')
print(historical_data['category_3'].value_counts())

print('\nValue counts for category features of New merchant Transactions :\n')
print(newmerchant_data['category_1'].value_counts())
print('*****************************')
print(newmerchant_data['category_2'].value_counts())
print('*****************************')
print(newmerchant_data['category_3'].value_counts())

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (15, 5));
historical_data['category_1'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='category_1', rot=0);
historical_data['category_2'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='category_2', rot=0);
historical_data['category_3'].value_counts().sort_index().plot(kind='bar', ax=ax[2], color='gold', title='category_3', rot=0);
plt.suptitle('Counts for category features of Historical Transactions New merchant Transactions');


fig, ax = plt.subplots(1, 3, figsize = (15, 5));
newmerchant_data['category_1'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='category_1', rot=0);
newmerchant_data['category_2'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='category_2', rot=0);
newmerchant_data['category_3'].value_counts().sort_index().plot(kind='bar', ax=ax[2], color='gold', title='category_3', rot=0);
plt.suptitle('Counts for category features of New merchant Transactions');

**Observation : No data drift**

The distribution of these three category features are almost 
identical in historical and new transactions.This shows these Category feature represent the inately charcterstics of the transactions which is constant over the period.  So, these features can be an importance feature in the decision function on final model.

**Distrbution of target over categorical features :**

**Note :** The train.csv file only has the target value, which is the feature we are gonna predict with models build in the future But, transactions data don't have the target values in it for each card_id's. By merging the "target" feature with the transactions data will help in Data analysis to fully understand different features in transactional dataFrame.

In [ ]:
train_data.shape, historical_data.shape, newmerchant_data.shape

In [ ]:
# Stable documentation : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html#pandas.merge
# merging target value of card_id for each transaction in historical_transactions Data
historical_data = pd.merge(historical_data, train_data[['card_id','target']], how = 'outer', on = 'card_id')

# merging target value of card_id for each transaction in new_merchants_transactions Data
newmerchant_data = pd.merge(newmerchant_data, train_data[['card_id','target']], how = 'outer', on = 'card_id')

In [ ]:
historical_data.shape, newmerchant_data.shape

In [ ]:
historical_data.head()

In [ ]:
newmerchant_data.head()

In [ ]:
plt.figure(figsize = (20,10))
plt.subplot(231)
sns.kdeplot(x ='target',data = historical_data,hue = 'category_1',palette='Dark2_r')
plt.title("Distribution of target over Category_1 in historical data")
plt.subplot(232)
sns.kdeplot(x ='target',data = historical_data,hue = 'category_2',palette='Dark2_r')
plt.title("Distribution of target over Category_2 in historical data")
plt.subplot(233)
sns.kdeplot(x ='target',data = historical_data,hue = 'category_3',palette='rainbow')
plt.title("Distribution of target over Category_3 in historical data")
plt.subplot(234)
sns.kdeplot(x ='target',data = newmerchant_data,hue = 'category_1',palette='Dark2_r')
plt.title("Distribution of target over Category_1 in new_merchent data")
plt.subplot(235)
sns.kdeplot(x ='target',data = newmerchant_data,hue = 'category_2',palette='Dark2_r')
plt.title("Distribution of target over Category_2 in new_merchent data")
plt.subplot(236)
sns.kdeplot(x ='target',data = newmerchant_data,hue = 'category_3',palette='rainbow')
plt.title("Distribution of target over Category_3 in new_merchent data")
plt.tight_layout()
plt.show()

**Observations :**

* These three category features doesn't explicity help to differentiate the target Score(Loyalty Score). Every category have outliers in each of the sub_categories. And Almost all the category have Same IQR range.

* These anonymous features doesn't reveal any important info for further feature engineering of these categories.

**Note:** The same information can be gathered by using box-plot and violin-plot, I have tried all of them. Here, I use kdeplot as I found it more visually appealing. In further analysis I have used Box-plot more often.


In [ ]:
# There are card_id without target
newmerchant_data.info(show_counts=True)

**Authorized Flag Feature Analysis :**

In [ ]:
print('Value counts for Authorized Flag of Historical Transactions :')
print(historical_data['authorized_flag'].value_counts())
print('*************************************************************')
print('Value counts for Authorized Flag of New Merchant Transactions :')
print(newmerchant_data['authorized_flag'].value_counts())

#barplot for the authorized_flag feature
fig, ax = plt.subplots(1, 2, figsize = (12, 5));
historical_data['authorized_flag'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='\nauthorized_flag(historical_transactions)');
newmerchant_data['authorized_flag'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='\n   authorized_flag(new_merchant_transactions)');

**Observations :**

* The new transactions have no "N" category in authorized_flag. This historical transactions have both "Y" and "N".

* The authorized_flag 'Y' if approved, 'N' if denied - whether the transaction is approved or Denied.

* If we calculate percentage of authorized transaction in historical transaction. At average 91.3545% transactions are authorized.

* This feature is an important feature for predicting the Loyalty score. because, if the card's transactions are approved most of time, there is a great chance the cards can have high Loyalty Score

Distributions of target over authorized flag :

In [ ]:
plt.figure(figsize = (14,5))
plt.subplot(121)
sns.boxplot(y = 'target',x= 'authorized_flag', data = historical_data)
plt.title("Distributions of target over authorized flag(historical_transactions)")
plt.subplot(122)
sns.boxplot(y = 'target',x= 'authorized_flag', data = newmerchant_data)
plt.title("Distributions of target over authorized flag(new_merchant_transactions)")
plt.tight_layout()
plt.show()

**Observations :** 

* The authorized Flag also doesn't give a suspectble change in the IQR range between authorized and un_authorized transactions.

* Even for the un_authorized transactions card users have same IQR. Because of the many transactions by an user, these un_authorized doesn't have much effect.

* But this categorical features also should be included using response coding (to represent the categorical data, probability of the data point belonging to a particular class given a category).

**Analysis of installments feature :**

In [ ]:
print('Quantile values for installments in Historical Transaction :')
print('25th Percentile :',historical_data['installments'].quantile(0.25))
print('50th Percentile :',historical_data['installments'].quantile(0.50))
print('75th Percentile :',historical_data['installments'].quantile(0.75))
print('100th Percentile :',historical_data['installments'].quantile(1))
print('\n******************************************************************\n')
print('Quantile values for installments in New Merchant Transaction :')
print('25th Percentile :',newmerchant_data['installments'].quantile(0.25))
print('50th Percentile :',newmerchant_data['installments'].quantile(0.50))
print('75th Percentile :',newmerchant_data['installments'].quantile(0.75))
print('100th Percentile :',newmerchant_data['installments'].quantile(1))

Distribution of target over installment feature :

In [ ]:
plt.figure(figsize = (14,5))
plt.subplot(121)
sns.boxplot(x=historical_data["installments"])
plt.title("Distributions of installments (historical_transactions)")
plt.subplot(122)
sns.boxplot(x=newmerchant_data["installments"])
plt.title("Distributions of installments (new_merchant_transactions)")
plt.tight_layout()
plt.show()

In [ ]:
historical_data["installments"].describe()

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(121)
sns.boxplot(y='target',x= 'installments', data = historical_data)
plt.title("Distrbutions of target over installments(historical_transactions)")
plt.subplot(122)
sns.boxplot(y='target',x = 'installments', data = newmerchant_data)
plt.title("Distrbutions of target over installments(new_merchant_transactions)")
plt.tight_layout()
plt.show()

**Observations :** The installments also have outliers, these outliers should be taken care in data preprocessing. In historical_transactions and new_merchants_transactions the 75% of installments are below 1. So, most of the payments through the cards are instant payments or short term installments.

**Analysis of purchase_amount feature :**

In [ ]:
print('Quantile values for purchase amount in Historical Transaction :')
print('25th Percentile :',historical_data['purchase_amount'].quantile(0.25))
print('50th Percentile :',historical_data['purchase_amount'].quantile(0.50))
print('75th Percentile :',historical_data['purchase_amount'].quantile(0.75))
print('100th Percentile :',historical_data['purchase_amount'].quantile(1))
print('\n******************************************************************\n')
print('Quantile values for purchase amount in New Merchant Transaction :')
print('25th Percentile :',newmerchant_data['purchase_amount'].quantile(0.25))
print('50th Percentile :',newmerchant_data['purchase_amount'].quantile(0.50))
print('75th Percentile :',newmerchant_data['purchase_amount'].quantile(0.75))
print('100th Percentile :',newmerchant_data['purchase_amount'].quantile(1))

**Observation :** The IQR range value is very small. And there is one outlier which have 6010603.9717525. These outlier can skew the final model performance. purchase_amount is normalized. Let's have a look at it nevertheless.

In [ ]:
plt.figure(figsize = (13,5))
plt.subplot(121)
plt.title('Purchase amount (Historical Transaction)');
historical_data['purchase_amount'].plot(kind='hist');
plt.subplot(122)
plt.title('Purchase amount (NewMerchant Transaction)');
newmerchant_data['purchase_amount'].plot(kind='hist');

In [ ]:
historical_data['purchase_amount'].value_counts()

In [ ]:
print('For purchase_amount in Historical transactions :')
for i in [-1, 0]:
    n = historical_data.loc[historical_data['purchase_amount'] < i].shape[0]
    print(f"There are {n} transactions with purchase_amount less than {i}.")
for i in [0, 10, 100]:
    n = historical_data.loc[historical_data['purchase_amount'] > i].shape[0]
    print(f"There are {n} transactions with purchase_amount more than {i}.")
    
print(70 * '-')

print('For purchase_amount in New Merchant transactions :')
for i in [-1, 0]:
    n = newmerchant_data.loc[newmerchant_data['purchase_amount'] < i].shape[0]
    print(f"There are {n} transactions with purchase_amount less than {i}.")
for i in [0, 10, 100]:
    n = newmerchant_data.loc[newmerchant_data['purchase_amount'] > i].shape[0]
    print(f"There are {n} transactions with purchase_amount more than {i}.")

**Observation :** As we can see the major chunk of transactions has purchase_amount less than 0. let us see Purchase amount distribution for negative values.

In [ ]:
plt.figure(figsize = (14,5))
plt.subplot(121)
plt.title(' Negative purchase_amount distribution (Historical)');
historical_data.loc[historical_data['purchase_amount'] < 0, 'purchase_amount'].plot(kind='hist');
plt.subplot(122)
plt.title('Negative purchase_amount distribution (New Merchant)');
newmerchant_data.loc[newmerchant_data['purchase_amount'] < 0, 'purchase_amount'].plot(kind='hist');

**Observation :** It seems that almost all transactions have purchase amount in range (-1, 0). Quite a strong normalization and high outliers, which will need to be processed.

Now, let's see purchase_amount feature over target variable :



In [ ]:
# Before removing detected outlier in 'historical_data'
historical_data.shape

In [ ]:
#There is one outlier which have value 6010603.9717525. We will remove it for further EDA.
historical_data = historical_data[historical_data['purchase_amount']  != 6010603.9717525]

In [ ]:
# After removal from 'historical_data'
historical_data.shape

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(121)
sns.scatterplot(data=historical_data, x="purchase_amount", y="target")
plt.title("purchase_amount (historical_transaction) over target")
plt.subplot(122)
sns.scatterplot(data=newmerchant_data, x="purchase_amount", y="target")
plt.title("purchase_amount (newmerchant_transaction) over target")
plt.tight_layout()
plt.show()

**Observations :**

* One key observation here is, Most of the outliers in target having value around -30 are having very less purchase amount.
* With the increase in purchase amount customer become more loyal, as target score increases.???  Target loyalty score decrease with more purchase amount???

**Analysis of feature Month_lag :**

In [ ]:
plt.figure(figsize = (13,5))
plt.subplot(121)
plt.title('Month lag (Historical Transaction)');
historical_data['month_lag'].plot(kind='hist');
plt.subplot(122)
plt.title('Month lag (NewMerchant Transaction)');
newmerchant_data['month_lag'].plot(kind='hist');

Distribution of target over month_lag feature :

In [ ]:
plt.figure(figsize = (14,5))
plt.subplot(121)
sns.boxplot(y= 'target',x= 'month_lag', data = historical_data)
plt.title("Distrbutions of target over month_lag (historical_transactions)")

plt.subplot(122)
sns.boxplot(y= 'target',x= 'month_lag', data = newmerchant_data)
plt.title("Distrbutions of target over month_lag (historical_transactions)")
plt.tight_layout()
plt.show()

**Observations :** 

* The Month_lag gives important info to predict the loyalty score. For a Purchase in installments, how many months the card lags from the actual end date of installment is the month_lag feature.

* The historical_transactions have month_lags from 0 to 13. which means the cards with transactions in histortical_transactions data have lag of installments from 0 to 13. But, the new_merchant_transactions have month_lag 1 and 2 only.

* This again proves the difference in the transactions type between the historical and new merchants.

In [ ]:
newmerchant_data['month_lag'].value_counts()

**Analysis of feature 'purchase_date' :**

At first, we convert purchase_date to datetime format :

In [ ]:
historical_data['purchase_date'] = pd.to_datetime(historical_data['purchase_date'],
                                                  format='%Y-%m-%d %H:%M:%S')
newmerchant_data['purchase_date'] = pd.to_datetime(newmerchant_data['purchase_date'],
                                                   format='%Y-%m-%d %H:%M:%S')

Number of transactions vs Year :

In [ ]:
#barplot for the Number of transactions vs Year
fig, ax = plt.subplots(1, 2, figsize = (14, 5));
historical_data['purchase_date'].dt.year.value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal',
                                                                          title='Transactions Vs Year (histortical_transactions)', rot=0)
newmerchant_data['purchase_date'].dt.year.value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown',
                                                                           title='Transactions Vs Year (new_merchant transactions)', rot=0)


print('Year-Wise Percentage distribution of purchase_date (Historical-Transaction) :')
print(historical_data['purchase_date'].dt.year.value_counts(normalize = True)*100)
print('\nYear-Wise Percentage distribution of purchase_date (NewMerchant-Transaction) :')
print(newmerchant_data['purchase_date'].dt.year.value_counts(normalize = True)*100)

**Observations :**

* In historical_transactions, The transactions with respect to year 2017 is way more (~82%) than transactions in 2018 (18%). 

* But, In new_merchant_transactions, transactions with respect to 2018 is way more (~85%) than transactions in 2018 (15%).

* Then we can say, new_merchant_transactions are the recent year transactions. This is the reason for the disparity in the purchase amount and installment features.

Number of transactions vs Week

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (15, 5));
historical_data['purchase_date'].dt.dayofweek.value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal',
                                                                               title='Transactions Vs dayofweek (histortical_transactions)', rot=0);
newmerchant_data['purchase_date'].dt.dayofweek.value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown',
                                                                                title='Transactions Vs dayofweek (new_merchant_transactions)', rot=0);

Distribution of target over dayofweek :

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(121)
sns.boxplot(y = historical_data['target'], x = historical_data['purchase_date'].dt.dayofweek)
plt.xticks(range(0,7),labels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
plt.xlabel('Days of week')
plt.title("Distribution of target over dayofweek (histortical_transactions)")

plt.subplot(122)
sns.boxplot(y = historical_data['target'], x = newmerchant_data['purchase_date'].dt.dayofweek)
plt.xticks(range(0,7),labels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
plt.xlabel('Days of week')
plt.title("Distribution of target over dayofweek (new_merchant_transactions)")
plt.tight_layout()
plt.show()

Number of transactions vs hour

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (15, 5));
historical_data['purchase_date'].dt.hour.value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal',
                                                                          title='Transactions Vs hour (histortical_transactions)', rot=0);
newmerchant_data['purchase_date'].dt.hour.value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown',
                                                                           title='Transactions Vs hour (new_merchant_transactions)', rot=0);

Distribution of target over hour :

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
sns.boxplot(y = historical_data['target'], x = historical_data['purchase_date'].dt.hour)
plt.xlabel('Hour')
plt.xticks(range(0,24))
plt.title("Distribution of target over hour (histortical_transactions)")

plt.subplot(122)
sns.boxplot(y = historical_data['target'], x = newmerchant_data['purchase_date'].dt.hour)
plt.xlabel('Hour')
plt.xticks(range(0,24))
plt.title("Distribution of target over hour (new_merchant_transactions)")
plt.tight_layout()
plt.show()


**Observations :**

* From the distribution of both weekly and hourly transactions count, these transactions have not much difference in their distributions.

* Since, the data given in the problem is a generated data and not a real time data. The distribution of the transactions over the purchase date is similar.

* But, the type of transactions differs from historical and new_merchants in terms of purchase_amount, month_lag and installments.

* By checking the number of merchants are in both historical and new_merchants transactions, we can get exclusive informations of the merchants.

**Let's create a feature called Number of transactions for each card_id and see - How it impacts target variable ?**

Number of Transactions feature is not explicitly given in any of the file but we can derive it with some hacks :

In [ ]:
# For historical transactions
g = historical_data[['card_id']].groupby('card_id')
df_transaction_counts = g.size().reset_index(name='num_transactions')
historical_data = pd.merge(historical_data ,df_transaction_counts, on="card_id",how='left')
historical_data.head()

In [ ]:
historical_data['num_transactions'].describe()

In [ ]:
#For New Merchant transaction
g = newmerchant_data[['card_id']].groupby('card_id')
df_transaction_counts = g.size().reset_index(name='num_transactions')
newmerchant_data = pd.merge(newmerchant_data ,df_transaction_counts, on="card_id",how='left')
newmerchant_data.head()

In [ ]:
newmerchant_data['num_transactions'].describe()

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(121)
sns.scatterplot(data=historical_data, x="num_transactions", y="target")
plt.title("Number of transactions (historical_transaction) VS target")
plt.subplot(122)
sns.scatterplot(data=newmerchant_data, x="num_transactions", y="target")
plt.title("Number of Transactions (newmerchant_transaction) VS target")
plt.tight_layout()
plt.show()

**Observations :**

* One key observation here is, Most of the outliers in target having value around -30 are having very less no of transactions.
* With increase in no of transactions customer become more loyal, as target score increases???

**Correlation between variables : Variance Inflation Factor**

In [ ]:
selected_columns = ['category_2','month_lag','purchase_amount','state_id','subsector_id', 'installments']
data_frame = newmerchant_data[selected_columns]

data_frame = data_frame.dropna()

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(data_frame.iloc[:,:].values, i) for i in range(data_frame.shape[1])]
vif["features"] = data_frame.columns
vif

All values are under 10, let's add some more features and again we'll calculate the VIF :




In [ ]:
Dict = {'A':1,'B':2,'C':3}
Dict1 = {'Y':1,'N':0}

selected_columns = ['authorized_flag','category_3','category_2','month_lag','purchase_amount','state_id','subsector_id', 'installments']
data_frame = newmerchant_data[selected_columns]
data_frame['category_3'] = data_frame['category_3'].map(Dict)
data_frame['authorized_flag'] = data_frame['authorized_flag'].map(Dict1)

data_frame = data_frame.dropna()

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(data_frame.iloc[:,:].values, i) for i in range(data_frame.shape[1])]
vif["features"] = data_frame.columns
vif

**Observations :**

* The value for the authorized flag is somewhat higher, it is around 32 which indicates possible correlation. So this variable needs further investigation.

* Other than the authorized flag the remaining variables doesn't look correlated. They are well under 2.

<h2>Exploring the Merchant Data :</h2>



In [ ]:
# Read excel-formatted data dictionary file with pandas
#data_dictionary = pd.read_excel('../input/elo-merchant-category-recommendation/Data_Dictionary.xlsx', sheet_name='merchant')
#data_dictionary

In [ ]:
merchants_data.head()

In [ ]:
merchants_data.info()

In [ ]:
merchants_data.isna().any()

**Observations :** Merchant data has missing values in columns : avg_sales_lag3, avg_sales_lag6 and avg_sales_lag12 

**Analysis of Numerical features : numerical_1 and numerical_2**

In [ ]:
print('Quantile values for numeric_1 in Transaction data:')
print('25th Percentile :',merchants_data['numerical_1'].quantile(0.25))
print('50th Percentile :',merchants_data['numerical_1'].quantile(0.50))
print('75th Percentile :',merchants_data['numerical_1'].quantile(0.75))
print('100th Percentile :',merchants_data['numerical_1'].quantile(1))
print('\n******************************************************************\n')
print('Quantile values for numeric_2 in Transaction data:')
print('25th Percentile :',merchants_data['numerical_2'].quantile(0.25))
print('50th Percentile :',merchants_data['numerical_2'].quantile(0.50))
print('75th Percentile :',merchants_data['numerical_2'].quantile(0.75))
print('100th Percentile :',merchants_data['numerical_2'].quantile(1))

**Observation :** I think the Distribution of numerical_1 and numerical_2 featurs are almost identical, because three quantiles have identical values.

In [ ]:
plt.figure(figsize=(12,5) )
plt.subplot(121)
sns.kdeplot(np.log10(merchants_data['numerical_1']),shade=True)
plt.title("PDF of numerical_1 in LogScale")
plt.xlabel('log (numerical_1)')
plt.subplot(122)
sns.kdeplot(np.log10(merchants_data['numerical_2']),shade=True)
plt.title("PDF of numerical_2 in LogScale")
plt.xlabel('log (numerical_2)')
plt.tight_layout()
plt.show()

**Observation :** After plotting PDF, it is very clear that both the features have same distribution, may be they are duplicates of each other.

**Note :** The values for numeric_1 and numeric_2 are mostly -ve and very near to zero. So, I preferred LogScale for analysis.

**Analysis of the three anonymized category features : category_1,category_2 and category_4**

In [ ]:
print('Value counts for category features of Merchants data :\n')
print(merchants_data['category_1'].value_counts())
print('******************************')
print(merchants_data['category_2'].value_counts())
print('******************************')
print(merchants_data['category_4'].value_counts())

fig, ax = plt.subplots(1, 3, figsize = (15, 5));
merchants_data['category_1'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal', title='category_1', rot=0);
merchants_data['category_2'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown', title='category_2', rot=0);
merchants_data['category_4'].value_counts().sort_index().plot(kind='bar', ax=ax[2], color='gold', title='category_3', rot=0);
plt.suptitle('Counts for category features of Merchants_data');

**Observation :** These are anonymous categories, which can represent some properties of the merchants, which is still unclear after merging with the transactions data it can reveal more info.

**Analysis of 'feature most_recent_sales_range' and 'most_recent_purchases_range' :**

In [ ]:
print(merchants_data['most_recent_sales_range'].value_counts())
print('*******************************************')
print(merchants_data['most_recent_purchases_range'].value_counts())

fig, ax = plt.subplots(1, 2, figsize = (10, 5));
merchants_data['most_recent_sales_range'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal',
                                                                           title='most_recent_sales_range', rot=0);
merchants_data['most_recent_purchases_range'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown',
                                                                               title='most_recent_purchases_range', rot=0);

**Observations :**

* Both the features have very similar distributions.

* The sales range in last active month is a categorical feature with "A","B","C","D","E". after observing the trend from graph we can say Range of revenue (monetary units) is in order E > D > C > B > A.

* The Bar Plot shows there are many merchants with revenue range of "E" than other ranges.

* And also, Bar Plot shows there are many merchants with purchase quantity range of "E" than other ranges.

* The sales range and purchase range can be used in aggregated to know the card_id's most visited merchants in the final features for training.

**Analysis of Sales Average features :
'avg_sales_lag3', 'avg_purchases_lag6', 'avg_sales_lag12', 'avg_purchases_lag3', 'avg_purchases_lag6', 'avg_purchases_lag12'**


In [ ]:
print('Quantile values for avg_sales_lag3 in Transaction data:')
print('25th Percentile :',merchants_data['avg_sales_lag3'].quantile(0.25))
print('50th Percentile :',merchants_data['avg_sales_lag3'].quantile(0.50))
print('75th Percentile :',merchants_data['avg_sales_lag3'].quantile(0.75))
print('100th Percentile :',merchants_data['avg_sales_lag3'].quantile(1))
print('\n******************************************************************\n')
print('Quantile values for avg_sales_lag6 in Transaction data:')
print('25th Percentile :',merchants_data['avg_sales_lag6'].quantile(0.25))
print('50th Percentile :',merchants_data['avg_sales_lag6'].quantile(0.50))
print('75th Percentile :',merchants_data['avg_sales_lag6'].quantile(0.75))
print('100th Percentile :',merchants_data['avg_sales_lag6'].quantile(1))
print('Quantile values for numeric_1 in Transaction data6:')
print('\n******************************************************************\n')
print('Quantile values for avg_sales_lag12 in Transaction data:')
print('25th Percentile :',merchants_data['avg_sales_lag12'].quantile(0.25))
print('50th Percentile :',merchants_data['avg_sales_lag12'].quantile(0.50))
print('75th Percentile :',merchants_data['avg_sales_lag12'].quantile(0.75))
print('100th Percentile :',merchants_data['avg_sales_lag12'].quantile(1))

In [ ]:
print('Statistical insights for avg_purchases_lag3 in Transaction data:')
print(merchants_data['avg_purchases_lag3'].describe())
print('\n******************************************************************\n')
print('Statistical insights for avg_purchases_lag6 in Transaction data:')
print(merchants_data['avg_purchases_lag6'].describe())
print('\n******************************************************************\n')
print('Statistical insights for avg_purchases_lag12 in Transaction data:')
print(merchants_data['avg_purchases_lag12'].describe())

**Observation :** There are outliers with the value inf in each of these columns, we have to deal with it. For EDA part, I am removing the corresponding rows with the inf values in the columns avg_purchases_lag3, avg_purchases_lag6, avg_purchases_lag12. We will see what else we can do with these outliers in preprocessing part.

In [ ]:
merchants_data = merchants_data[merchants_data['avg_purchases_lag3']  != np.inf]
merchants_data = merchants_data[merchants_data['avg_purchases_lag6']  != np.inf]
merchants_data = merchants_data[merchants_data['avg_purchases_lag12']  != np.inf]

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(231)
sns.kdeplot(np.log10(merchants_data['avg_sales_lag3']),shade=True)
plt.title("PDF of avg_sales_lag3 in LogScale")
plt.xlabel('log(avg_sales_lag3)')
plt.subplot(232)
sns.kdeplot(np.log10(merchants_data['avg_sales_lag6']),shade=True)
plt.title("PDF of avg_sales_lag6 in LogScale")
plt.xlabel('log(avg_sales_lag6)')
plt.subplot(233)
sns.kdeplot(np.log10(merchants_data['avg_sales_lag12']),shade=True)
plt.title("PDF of avg_sales_lag12 in LogScale")
plt.xlabel('log(avg_sales_lag12)')
plt.subplot(234)
sns.kdeplot(np.log10(merchants_data['avg_purchases_lag3']),shade=True)
plt.title("PDF of avg_purchases_lag3 in LogScale")
plt.xlabel('log(avg_purchases_lag3)')
plt.subplot(235)
sns.kdeplot(np.log10(merchants_data['avg_purchases_lag6']),shade=True)
plt.title("PDF of avg_purchases_lag6 in LogScale")
plt.xlabel('log(avg_purchases_lag6)')
plt.subplot(236)
sns.kdeplot(np.log10(merchants_data['avg_purchases_lag12']),shade=True)
plt.title("PDF of avg_purchases_lag12 in LogScale")
plt.xlabel('log(avg_purchases_lag12)')
plt.tight_layout()
plt.show()

**Observations :** 

* The average purchases and sales across 3, 6 and 12 months are distributed near 1.

* And, there are outliers in all the average sales and purchases. These features gives info about the merchants but not about the card_id's. The information about the merchants have to cumulated for each card_id's.

**Note :** The values for All the sales features listed above are mostly surrounded very near to 1. So, I preferred LogScale for analysis.

**Quantity of active months : Analysis of features ('active_months_lag3', 'active_months_lag6' and 'active_months_lag12') :**

In [ ]:
print(merchants_data['active_months_lag3'].value_counts())
print('**********************************')
print(merchants_data['active_months_lag6'].value_counts())
print('**********************************')
print(merchants_data['active_months_lag12'].value_counts())

fig, ax = plt.subplots(1, 3, figsize = (15, 5));
merchants_data['active_months_lag3'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal',
                                                                      title='active_months_lag3', rot=0);
merchants_data['active_months_lag6'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown',
                                                                      title='active_months_lag6', rot=0);
merchants_data['active_months_lag12'].value_counts().sort_index().plot(kind='bar', ax=ax[2], color='gold',
                                                                       title='active_months_lag12', rot=0);
plt.suptitle('Counts of Active month lags');

**Observations :** The active months features are greatly skewed and doesn't provide any vital information about the cards.

**Correlation between variables : Variance Inflation Factor**

In [ ]:
selected_columns = ['numerical_1', 'numerical_2','category_2','avg_sales_lag3','avg_sales_lag6','avg_sales_lag12','avg_purchases_lag3','avg_purchases_lag6','avg_purchases_lag12','active_months_lag3','active_months_lag6','active_months_lag12']
data_frame = merchants_data[selected_columns]
data_frame = data_frame.dropna()

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(data_frame.iloc[:,:].values, i) for i in range(data_frame.shape[1])]
vif["features"] = data_frame.columns
vif

**Observation :** Looks like there are variables which are heavily correlated like 'active_months_lag6', 'avg_purchase_lag6' and 'avg_sales_lag_6' and 'avg_purchase_lag12' and as we seen before the 'numerical_1' and 'numerical_2' have similar values and distributions and they are correlated.



**TOTAL OBSERVATIONS :**

1) Target variable i.e. Loyalty scores are real-numbers, It directly gives us the intuition that we have to go for a supervised machine learning regression model to solve this problem.

2) The data files are train, test, new_merchant, merchant and historical transactions. but datasets are largely anonymized, and the meaning of the features are not elaborated.

3) The dimensionality of train and test data is very less. That clearly shows that the information provided is not sufficient for training. As only three features have been given in the train file which seems to be not sufficient to make good predictions. More features must be added to this with the help of domain knowledge and the business problem given.

4) Distribution of both the train and test are almost identical. So there is no time based splitting in the make over of the data. And, it assures for prediction of the test data.

5) The target variable is normally distributed but, there are outliers which seems to be accumulated around -30.

6) Data is not complete as nan values are present in the merchants, historical and new merchants transactions, so these missing values must be imputed for better predciton.

7) One-hot encoding/response coding of categorical features should be done for better prediction. The categorical features present across dataset are large in number than numerical features. 

8) Merchants data have high number of correlated features in it as compared to other data files. This is suggested by the calcuation of the VIF Scores 

9) The time features can reveal the inherent property of the transactions and the transactions are time dependent, the engineered features from the features like purchase_date will be useful in prediction.

10) In the historical transactions data there is this feature called 'authorized_flag' count which indicates whether the transaction is authorized or not. There is very less number of transactions which is not authorized. Considering this flag features as a separater in the feature engineering can results can give better prediction.

At the End of the Exploration of the transactions, merchants and train data, the given features of transactions are not big factor for the calculation of the target Score.

There exist an aggregrated or engineered feature or features which can be helpful in predicting the target Score.

With the different feature engineering techniques and market research techniques we have to produce the new features which may or may not be very useful in the prediction model.

By implementing the major feature engineering ideas we have to produce features and build model upon it.